# Building NNs with LLMs 

Import Required Libraries

In [5]:
import requests
import json
import numpy as np
from IPython.display import Markdown, display, Code
import pandas as pd
import re
import json
from datetime import datetime, timedelta
!pip install tensorflow_recommenders
!pip install keras-tuner


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


Helper Functions for LLM Communication

In [25]:
api_key='H_ClMkEoDcpQkDU4iYaTd3UNMuUUdPDbeQbDVHMnGms'

In [26]:
def get_resp_oai(input_text, model):
    url = "https://llm.api.ai8.io/query_llm"
    data = {
        # Specify the model that you want to use
        "model": model,
        "messages": [
                    {"role": "system", "content": "You act as a highly intelligent system. Your job is to analyse the movie data and predict the rating scores of movies considering various movie features"},
                    {"role": "user", "content": input_text}
        ]
    }
    headers = {'Authorization': api_key}
    response = requests.post(url, json=data, headers=headers)
    if response.status_code == 200:
        response_data = json.loads(response.content)
        model_response = extract_message_oai(response_data)
        return model_response
    else:
        return {"statusCode": response.status_code, "body": response.content}

def extract_message_oai(response_data):
    message_content = response_data.get("choices", [])[0].get("message", {}).get("content", "")
    # format the extracted message as markdown
    markdown_content = "---\n\n" + message_content + "\n\n---"
    return markdown_content

Inspect data: to create effective propmts

In [27]:
import pandas as pd
final_movies = pd.read_csv("final_merged_df.csv")
final_movies.head()

,Unnamed: 0,director_name,duration,actor_2_name,genres_x,actor_1_name,movie_title,actor_3_name,movie_imdb_link,language,...,Genre_Western,duration_category,rating_category,budget_category,revenue_category,budget_revenue_ratio,log_budget,log_revenue,log_title_year,sqr_root_duration
0,0,James Cameron,178.0,Joel David Moore,Action|Adventure|Fantasy|Sci-Fi,CCH Pounder,Avatar,Wes Studi,http://www.imdb.com/title/tt0499549/?ref_=fn_t...,English,...,0,165-180 Minutes,07-Aug,Greater than 200 Million,Greater than 1.8 Billion,0.085009,19.283571,21.748578,7.605890,13.341664
1,1,Gore Verbinski,169.0,Orlando Bloom,Action|Adventure|Fantasy,Johnny Depp,Pirates of the Caribbean At Worlds End,Jack Davenport,http://www.imdb.com/title/tt0449088/?ref_=fn_t...,English,...,0,165-180 Minutes,07-Aug,Greater than 200 Million,800-1000 Million,0.312176,19.519293,20.683485,7.604894,13.000000
2,2,Sam Mendes,148.0,Rory Kinnear,Action|Adventure|Thriller,Christoph Waltz,Spectre,Stephanie Sigman,http://www.imdb.com/title/tt2379713/?ref_=fn_t...,English,...,0,135-150 Minutes,06-Jul,Greater than 200 Million,800-1000 Million,0.278197,19.316769,20.596199,7.608871,12.165525
3,3,Christopher Nolan,164.0,Christian Bale,Action|Thriller,Tom Hardy,The Dark Knight Rises,Joseph Gordon-Levitt,http://www.imdb.com/title/tt1345836/?ref_=fn_t...,English,...,0,150-165 Minutes,08-Sep,Greater than 200 Million,1-1.20 Billion,0.230429,19.336971,20.804790,7.607381,12.806248
4,4,Andrew Stanton,132.0,Samantha Morton,Action|Adventure|Sci-Fi,Daryl Sabara,John Carter,Polly Walker,http://www.imdb.com/title/tt0401729/?ref_=fn_t...,English,...,0,120-135 Minutes,06-Jul,Greater than 200 Million,200-400 Million,0.915046,19.376192,19.464974,7.607381,11.489125


In [28]:
final_movies.columns

Index(['Unnamed: 0', 'director_name', 'duration', 'actor_2_name', 'genres_x',
       'actor_1_name', 'movie_title', 'actor_3_name', 'movie_imdb_link',
       'language', 'country', 'title_year', 'imdb_score', 'budget',
       'original_title', 'revenue', 'tagline', 'Genre_Action',
       'Genre_Adventure', 'Genre_Animation', 'Genre_Biography', 'Genre_Comedy',
       'Genre_Crime', 'Genre_Documentary', 'Genre_Drama', 'Genre_Family',
       'Genre_Fantasy', 'Genre_Film-Noir', 'Genre_History', 'Genre_Horror',
       'Genre_Music', 'Genre_Musical', 'Genre_Mystery', 'Genre_News',
       'Genre_Romance', 'Genre_Sci-Fi', 'Genre_Sport', 'Genre_Thriller',
       'Genre_War', 'Genre_Western', 'duration_category', 'rating_category',
       'budget_category', 'revenue_category', 'budget_revenue_ratio',
       'log_budget', 'log_revenue', 'log_title_year', 'sqr_root_duration'],
      dtype='object')

Helper Functions to Define Prompts 

In [17]:
# Funtion to define the main task : prompt 1
def define_prompt():
    
    prompt = """

Task Description:
Develop a neural network model using TensorFlow Recommenders to predict movie ratings based on the provided dataset. Your task is to design and implement a TensorFlow Recommenders model that accurately predicts movie ratings.

Dataset:
- Format: The dataset is named as 'final_movies' and is read in a CSV format with rows for each movie and columns containing information about movies. It consists of 4144 rows and 49 columns. This dataset provides a comprehensive set of features related to movies, including director, actors, genres, budget, revenue, and IMDb ratings, among others.

- Columns:
1. Unnamed: 0: Index column.
2. director_name: Name of the movie director.
3. duration: Duration of the movie in minutes.
4. actor_2_name: Name of the second actor in the movie.
5. genres_x: Genres of the movie.
6. actor_1_name: Name of the lead actor in the movie.
7. movie_title: Title of the movie.
8. actor_3_name: Name of the third actor in the movie.
9. movie_imdb_link: IMDb link of the movie.
10. language: Language of the movie.
11. country: Country where the movie was produced.
12. title_year: Year of release of the movie.
13. imdb_score: IMDb rating of the movie.
14. budget: Budget of the movie.
15. original_title: Original title of the movie.
16. revenue: Revenue generated by the movie.
17. tagline: Tagline of the movie.
18. Genre_Action: Binary indicator for Action genre.
19. Genre_Adventure: Binary indicator for Adventure genre.
20. Genre_Animation: Binary indicator for Animation genre.
21. Genre_Biography: Binary indicator for Biography genre.
22. Genre_Comedy: Binary indicator for Comedy genre.
23. Genre_Crime: Binary indicator for Crime genre.
24. Genre_Documentary: Binary indicator for Documentary genre.
25. Genre_Drama: Binary indicator for Drama genre.
26. Genre_Family: Binary indicator for Family genre.
27. Genre_Fantasy: Binary indicator for Fantasy genre.
28. Genre_Film-Noir: Binary indicator for Film-Noir genre.
29. Genre_History: Binary indicator for History genre.
30. Genre_Horror: Binary indicator for Horror genre.
31. Genre_Music: Binary indicator for Music genre.
32. Genre_Musical: Binary indicator for Musical genre.
33. Genre_Mystery: Binary indicator for Mystery genre.
34. Genre_News: Binary indicator for News genre.
35. Genre_Romance: Binary indicator for Romance genre.
36. Genre_Sci-Fi: Binary indicator for Sci-Fi genre.
37. Genre_Sport: Binary indicator for Sport genre.
38. Genre_Thriller: Binary indicator for Thriller genre.
39. Genre_War: Binary indicator for War genre.
40. Genre_Western: Binary indicator for Western genre.
41. duration_category: Categorized duration of the movie.
42. rating_category: Categorized IMDb rating of the movie.
43. budget_category: Categorized budget of the movie.
44. revenue_category: Categorized revenue of the movie.
45. budget_revenue_ratio: Ratio of budget to revenue.
46. log_budget: Logarithm of the budget.
47. log_revenue: Logarithm of the revenue.
48. log_title_year: Logarithm of the title year.
49. sqr_root_duration: Square root of the duration.

- To give you a bit more context about the dataset, and its features, this is the output we get after performing .info() method of the pandas library on the dataset (which gives more details about the dataset and it’s features): 

RangeIndex: 4144 entries, 0 to 4143
Data columns (total 49 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            4144 non-null   int64  
 1   director_name         4144 non-null   object 
 2   duration              4142 non-null   float64
 3   actor_2_name          4140 non-null   object 
 4   genres_x              4144 non-null   object 
 5   actor_1_name          4141 non-null   object 
 6   movie_title           4144 non-null   object 
 7   actor_3_name          4135 non-null   object 
 8   movie_imdb_link       4144 non-null   object 
 9   language              4137 non-null   object 
 10  country               4144 non-null   object 
 11  title_year            4144 non-null   float64
 12  imdb_score            4144 non-null   float64
 13  budget                4144 non-null   int64  
 14  original_title        4144 non-null   object 
 15  revenue               4144 non-null   float64
 16  tagline               3584 non-null   object 
 17  Genre_Action          4144 non-null   int64  
 18  Genre_Adventure       4144 non-null   int64  
 19  Genre_Animation       4144 non-null   int64  
 20  Genre_Biography       4144 non-null   int64  
 21  Genre_Comedy          4144 non-null   int64  
 22  Genre_Crime           4144 non-null   int64  
 23  Genre_Documentary     4144 non-null   int64  
 24  Genre_Drama           4144 non-null   int64  
 25  Genre_Family          4144 non-null   int64  
 26  Genre_Fantasy         4144 non-null   int64  
 27  Genre_Film-Noir       4144 non-null   int64  
 28  Genre_History         4144 non-null   int64  
 29  Genre_Horror          4144 non-null   int64  
 30  Genre_Music           4144 non-null   int64  
 31  Genre_Musical         4144 non-null   int64  
 32  Genre_Mystery         4144 non-null   int64  
 33  Genre_News            4144 non-null   int64  
 34  Genre_Romance         4144 non-null   int64  
 35  Genre_Sci-Fi          4144 non-null   int64  
 36  Genre_Sport           4144 non-null   int64  
 37  Genre_Thriller        4144 non-null   int64  
 38  Genre_War             4144 non-null   int64  
 39  Genre_Western         4144 non-null   int64  
 40  duration_category     4142 non-null   object 
 41  rating_category       4144 non-null   object 
 42  budget_category       3430 non-null   object 
 43  revenue_category      3126 non-null   object 
 44  budget_revenue_ratio  3552 non-null   float64
 45  log_budget            4144 non-null   float64
 46  log_revenue           4144 non-null   float64
 47  log_title_year        4144 non-null   float64
 48  sqr_root_duration     4142 non-null   float64
dtypes: float64(9), int64(25), object(15)
memory usage: 1.5+ MB

Dataset head in a JSON format: 
'{"Unnamed: 0":{"0":0,"1":1,"2":2,"3":3,"4":4},"director_name":{"0":"James Cameron","1":"Gore Verbinski","2":"Sam Mendes","3":"Christopher Nolan","4":"Andrew Stanton"},"duration":{"0":178.0,"1":169.0,"2":148.0,"3":164.0,"4":132.0},"actor_2_name":{"0":"Joel David Moore","1":"Orlando Bloom","2":"Rory Kinnear","3":"Christian Bale","4":"Samantha Morton"},"genres_x":{"0":"Action|Adventure|Fantasy|Sci-Fi","1":"Action|Adventure|Fantasy","2":"Action|Adventure|Thriller","3":"Action|Thriller","4":"Action|Adventure|Sci-Fi"},"actor_1_name":{"0":"CCH Pounder","1":"Johnny Depp","2":"Christoph Waltz","3":"Tom Hardy","4":"Daryl Sabara"},"movie_title":{"0":"Avatar","1":"Pirates of the Caribbean At Worlds End","2":"Spectre","3":"The Dark Knight Rises","4":"John Carter"},"actor_3_name":{"0":"Wes Studi","1":"Jack Davenport","2":"Stephanie Sigman","3":"Joseph Gordon-Levitt","4":"Polly Walker"},"movie_imdb_link":{"0":"http:\\/\\/www.imdb.com\\/title\\/tt0499549\\/?ref_=fn_tt_tt_1","1":"http:\\/\\/www.imdb.com\\/title\\/tt0449088\\/?ref_=fn_tt_tt_1","2":"http:\\/\\/www.imdb.com\\/title\\/tt2379713\\/?ref_=fn_tt_tt_1","3":"http:\\/\\/www.imdb.com\\/title\\/tt1345836\\/?ref_=fn_tt_tt_1","4":"http:\\/\\/www.imdb.com\\/title\\/tt0401729\\/?ref_=fn_tt_tt_1"},"language":{"0":"English","1":"English","2":"English","3":"English","4":"English"},"country":{"0":"USA","1":"USA","2":"UK","3":"USA","4":"USA"},"title_year":{"0":2009.0,"1":2007.0,"2":2015.0,"3":2012.0,"4":2012.0},"imdb_score":{"0":7.9,"1":7.1,"2":6.8,"3":8.5,"4":6.6},"budget":{"0":237000000,"1":300000000,"2":245000000,"3":250000000,"4":260000000},"original_title":{"0":"Avatar","1":"Pirates of the Caribbean At Worlds End","2":"Spectre","3":"The Dark Knight Rises","4":"John Carter"},"revenue":{"0":2787965087.0,"1":961000000.0,"2":880674609.0,"3":1084939099.0,"4":284139100.0},"tagline":{"0":"Enter the World of Pandora.","1":"At the end of the world, the adventure begins.","2":"A Plan No One Escapes","3":"The Legend Ends","4":"Lost in our world, found in another."},"Genre_Action":{"0":1,"1":1,"2":1,"3":1,"4":1},"Genre_Adventure":{"0":1,"1":1,"2":1,"3":0,"4":1},"Genre_Animation":{"0":0,"1":0,"2":0,"3":0,"4":0},"Genre_Biography":{"0":0,"1":0,"2":0,"3":0,"4":0},"Genre_Comedy":{"0":0,"1":0,"2":0,"3":0,"4":0},"Genre_Crime":{"0":0,"1":0,"2":0,"3":0,"4":0},"Genre_Documentary":{"0":0,"1":0,"2":0,"3":0,"4":0},"Genre_Drama":{"0":0,"1":0,"2":0,"3":0,"4":0},"Genre_Family":{"0":0,"1":0,"2":0,"3":0,"4":0},"Genre_Fantasy":{"0":1,"1":1,"2":0,"3":0,"4":0},"Genre_Film-Noir":{"0":0,"1":0,"2":0,"3":0,"4":0},"Genre_History":{"0":0,"1":0,"2":0,"3":0,"4":0},"Genre_Horror":{"0":0,"1":0,"2":0,"3":0,"4":0},"Genre_Music":{"0":0,"1":0,"2":0,"3":0,"4":0},"Genre_Musical":{"0":0,"1":0,"2":0,"3":0,"4":0},"Genre_Mystery":{"0":0,"1":0,"2":0,"3":0,"4":0},"Genre_News":{"0":0,"1":0,"2":0,"3":0,"4":0},"Genre_Romance":{"0":0,"1":0,"2":0,"3":0,"4":0},"Genre_Sci-Fi":{"0":1,"1":0,"2":0,"3":0,"4":1},"Genre_Sport":{"0":0,"1":0,"2":0,"3":0,"4":0},"Genre_Thriller":{"0":0,"1":0,"2":1,"3":1,"4":0},"Genre_War":{"0":0,"1":0,"2":0,"3":0,"4":0},"Genre_Western":{"0":0,"1":0,"2":0,"3":0,"4":0},"duration_category":{"0":"165-180 Minutes","1":"165-180 Minutes","2":"135-150 Minutes","3":"150-165 Minutes","4":"120-135 Minutes"},"rating_category":{"0":"7-8","1":"7-8","2":"6-7","3":"8-9","4":"6-7"},"budget_category":{"0":"Greater than 200 Million","1":"Greater than 200 Million","2":"Greater than 200 Million","3":"Greater than 200 Million","4":"Greater than 200 Million"},"revenue_category":{"0":"Greater than 1.8 Billion","1":"800-1000 Million","2":"800-1000 Million","3":"1-1.20 Billion","4":"200-400 Million"},"budget_revenue_ratio":{"0":0.0850092381,"1":0.3121758179,"2":0.2781968257,"3":0.2304286804,"4":0.9150457791},"log_budget":{"0":19.2835707033,"1":19.519293036,"2":19.3167687726,"3":19.3369714798,"4":19.3761921928},"log_revenue":{"0":21.7485778075,"1":20.683484968,"2":20.596198774,"3":20.8047896933,"4":19.4649744685},"log_title_year":{"0":7.6058900011,"1":7.6048944808,"2":7.6088706292,"3":7.6073814256,"4":7.6073814256},"sqr_root_duration":{"0":13.3416640641,"1":13.0,"2":12.1655250606,"3":12.8062484749,"4":11.4891252931}}'

Objectives:
* Utilise TensorFlow and TensorFlow Recommenders to build a neural network model.
* The model should take relevant features from the dataset as input and predict movie ratings.
* Experiment with different architectures, hyperparameters, and preprocessing techniques to optimise the model's performance.
* Evaluate the model using appropriate metrics and provide insights into its performance.
* Provide insights into the model's performance and potential areas for improvement.

Deliverables:
1. Python code implementing the TensorFlow Recommenders model for movie rating prediction.
2. A paragraph in comments documenting the model development process, including:
    * Description of the chosen architecture and rationale behind it.
    * Explanation of preprocessing techniques applied to the dataset.
    * Summary of experiments conducted to optimise the model's performance.
    * Evaluation metrics used and results obtained from model evaluation.
    * Insights into the model's performance and potential areas for improvement.

Requirements:
1. Use TensorFlow and tensorflow_recommenders library to build the model.
2. Include relevant features from the dataset as input to the model.
3. Experiment with at least two different model architectures.
4. Perform preprocessing on the dataset as necessary for model training.
5. Document the code clearly, including comments where necessary for clarity.
6. Submit the Python code and in an executable format (to be then executed with the function ‘exec()’).
    """
    return prompt

In [18]:
# function to convert response string into an executable format: prompt 3, 5
def helper_extraction_prompt(resp_str):
    return f"Can you transform this string: {resp_str} into an executable string by extracting the python code and keeping the rest of the information as comments? Please place comment symbols where nessesary and keep in mind that the given response string will be executed in a python code chunk using the function :'exec(given_response_string)."

In [19]:
# function to prompt the model for error resolution purposes given the error encountered and previous response as inputs: prompt 2,4
def prompt_error(prev_response, err):
    return f"Given that the code you provided: {prev_response} gives this error: {err}, help me resolve it by changing the code you provided where nessesary, your reposne should be in an executable code format"

### Prompt the Model and Display Responses

In [20]:
prompt1 = define_prompt()
msg_1 = get_resp_oai(prompt1,"gpt-4-0125-preview")
display(Markdown("<div style='color: #34568B;'>\n\n" + msg_1))

<div style='color: #34568B;'>

---

Given the task requirements and constraints, we can outline a potential solution for building a TensorFlow Recommenders model for predicting movie ratings. Due to this environment not supporting direct code execution or access to external libraries such as TensorFlow, I'll provide a detailed conceptual walkthrough and code blueprint that aligns with your deliverables and requirements.

### Data Preprocessing
Before moving to model building, it's important to preprocess the data:
1. **Handle missing values** - Decide whether to fill missing values (e.g., with median or average for numerical columns and a placeholder value or the most common value for categorical columns) or to drop rows/columns with missing values.
2. **Normalize numerical features** - Use MinMaxScaler or StandardScaler for features like duration, budget, and revenue.
3. **Encode categorical features** - Use one-hot encoding or TensorFlow feature embedding for categorical fields such as director_name, actor names, genres, and country.

### TensorFlow Recommenders Model Blueprint

#### Import Necessary Libraries
```python
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Concatenate, Flatten
from tensorflow.keras.optimizers import Adam
```

#### Model Architecture #1: Basic Embedding Model
This example constructs a basic model that embeds categorical movie features and processes numerical features, aiming to predict movie ratings.

##### Feature Engineering and Model Definition
```python
# Assuming the preprocessing steps are applied, and data is split into train and test sets.

# Define input layers for each feature.
# Note: The number of unique values and embedding dimensions should be adjusted based on the actual data.
director_input = Input(shape=(1,), name="director_name")
director_embedding = Embedding(num_directors, embedding_dim)(director_input)
director_vec = Flatten(name="Flatten_Directors")(director_embedding)

# Repeat similar blocks for other categorical features (e.g., actors, genres_x)

# Numerical inputs do not need embedding
duration_input = Input(shape=(1,), name="duration")

# Combine all feature inputs
concat = Concatenate()([director_vec, /* other feature vectors, */, duration_input])

# Fully connected layers
fc = Dense(128, activation='relu')(concat)
fc = Dense(32, activation='relu')(fc)

# Output layer
output = Dense(1, activation='linear')(fc)

# Define model
model = Model(inputs=[director_input, /* other inputs */], outputs=output)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['mae'])
```

#### Training and Evaluation
```python
# Assuming X_train, y_train, X_test, y_test are prepared with appropriate feature columns
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

evaluation = model.evaluate(X_test, y_test)
print(f"Test Loss: {evaluation[0]}, Test MAE: {evaluation[1]}")
```

#### Model Architecture #2: Advanced Model with Mixed Features
For a more advanced model, consider including feature crosses, deeper embedding layers for high-cardinality features, and possibly recurrent architectures for handling sequence data (e.g., the sequence of actors might provide some contextual clue).

### Experimental Summary (Hypothetical)
Through experimentation, we might find that:
- Higher embedding dimensions for directors and genres lead to better performance but increase the risk of overfitting.
- Regularization techniques (such as dropout) are crucial to manage overfitting, especially with a complex model.
- Normalizing numerical features generally results in better performance.
- Model performance might vary significantly with different optimizers and learning rates, where Adam and a learning rate scheduler can be quite effective.

### Evaluation and Insights
We would use Mean Absolute Error (MAE) or Root Mean Squared Error (RMSE) as our main metrics to evaluate the model's performance due to the regression nature of the problem. Insights might reveal the importance of certain features over others and indicate potential data quality issues or areas where more data could enhance model accuracy.

#### Potential Areas for Improvement
- Incorporating more sophisticated feature engineering (e.g., polynomial features for numerical data).
- Exploration of more complex model architectures or ensemble methods.
- Extensive hyperparameter tuning and cross-validation to optimize model parameters effectively.

```python
# Reminder: This is a hypothetical implementation blueprint. Tailor model architectures, feature engineering, and training procedures to your specific dataset and use case.
```

Given the high-level overview and codes snippets, you should be able to implement a TensorFlow Recommenders model that learns from the movie dataset to predict ratings accurately. Remember, iteration over different model architectures and hyperparameters is key to finding the best performing model.

---

Execute the code given in the response message using`exec()`, minimal syntax manipulation required.

In [21]:
exec(msg_1[15:len(msg_1)-338].replace("path_to_final_movies.csv", "final_merged_df.csv").replace("genre_Action", "Genre_Action").replace("genre_Adventure", "Genre_Adventure").replace("genre_Animation", "Genre_Animation"))


SyntaxError: unterminated string literal (detected at line 1) (<string>, line 1)

Error Resultion Prompts

In [69]:
err = """ 
<string>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
Epoch 1/10
---------------------------------------------------------------------------
NotImplementedError                       Traceback (most recent call last)
Cell In[48], line 1
----> 1 exec(msg_1[15:len(msg_1)-338].replace("path_to_final_movies.csv", "final_merged_df.csv").replace("genre_Action", "Genre_Action").replace("genre_Adventure", "Genre_Adventure").replace("genre_Animation", "Genre_Animation"))

File <string>:73

File /usr/local/lib/python3.11/site-packages/keras/src/utils/traceback_utils.py:70, in filter_traceback.<locals>.error_handler(*args, **kwargs)
     67     filtered_tb = _process_traceback_frames(e.__traceback__)
     68     # To get the full stack trace, call:
     69     # `tf.debugging.disable_traceback_filtering()`
---> 70     raise e.with_traceback(filtered_tb) from None
     71 finally:
     72     del filtered_tb

File /tmp/__autograph_generated_filejo7rd7xr.py:15, in outer_factory.<locals>.inner_factory.<locals>.tf__train_function(iterator)
     13 try:
     14     do_return = True
---> 15     retval_ = ag__.converted_call(ag__.ld(step_function), (ag__.ld(self), ag__.ld(iterator)), None, fscope)
     16 except:
     17     do_return = False

File /usr/local/lib/python3.11/site-packages/tensorflow_recommenders/models/base.py:68, in Model.train_step(self, inputs)
     65 Custom train step using the `compute_loss` method.
     67 with tf.GradientTape() as tape:
---> 68   loss = self.compute_loss(inputs, training=True)
     70   # Handle regularization losses as well.
     71   regularization_loss = sum(self.losses)

File /usr/local/lib/python3.11/site-packages/tensorflow_recommenders/models/base.py:61, in Model.compute_loss(self, inputs, training)
     49 def compute_loss(self, inputs, training: bool = False) -> tf.Tensor:
     50   Defines the loss function.
     51 
     52   Args:
   (...)
     58     Loss tensor.
     59   
---> 61   raise NotImplementedError(
     62       "Implementers must implement the `compute_loss` method.")

NotImplementedError: in user code:

    File "/usr/local/lib/python3.11/site-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.11/site-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.11/site-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.11/site-packages/tensorflow_recommenders/models/base.py", line 68, in train_step
        loss = self.compute_loss(inputs, training=True)
    File "/usr/local/lib/python3.11/site-packages/tensorflow_recommenders/models/base.py", line 61, in compute_loss
        raise NotImplementedError(

    NotImplementedError: Implementers must implement the `compute_loss` method.
"""

prev_response = """
# Due to the constraints of this environment, I'm unable to run actual code or access external libraries such as TensorFlow.
# However, I can provide a guided example code that you can run in your local environment.

# Import necessary libraries
import pandas as pd
import tensorflow as tf
import tensorflow_recommenders as tfrs

# Load the dataset
df = pd.read_csv('path_to_final_movies.csv') # Replace path_to_final_movies.csv with the actual path of your dataset

# Basic preprocessing
# Fill missing values if necessary. For the purpose of this example, we'll assume minimal preprocessing.
df['duration'].fillna(df['duration'].mean(), inplace=True)

# Feature engineering: Select features for the model
# For simplicity, we'll use a subset of all available features.
features = ['duration', 'genre_Action', 'genre_Adventure', 'genre_Animation', 'budget', 'revenue', 'imdb_score']
df_model = df[features]

# Normalize continuous features
for feature in ['duration', 'budget', 'revenue']:
    df_model[feature] = (df_model[feature] - df_model[feature].mean()) / df_model[feature].std()

# Split the dataset into training and testing sets
train = df_model.sample(frac=0.8, random_state=42)
test = df_model.drop(train.index)

# Convert Pandas dataframes to TensorFlow datasets
train_ds = tf.data.Dataset.from_tensor_slices((dict(train.drop('imdb_score', axis=1)), train['imdb_score']))
test_ds = tf.data.Dataset.from_tensor_slices((dict(test.drop('imdb_score', axis=1)), test['imdb_score']))

train_ds = train_ds.batch(32).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.batch(32).prefetch(tf.data.AUTOTUNE)

# Define the model using TensorFlow Recommenders
class MovieRatingModel(tfrs.models.Model):

    def __init__(self):
        super().__init__()
        self.embedding_dimension = 32
        
        # Feature layers for various inputs
        self.duration_model = tf.keras.Sequential([
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(32, activation='relu')
        ])
        
        # Final task model: Prediction of IMDb score
        self.rating_prediction = tf.keras.Sequential([
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(1)
        ])
        
    def call(self, inputs):
        duration_features = self.duration_model(inputs['duration'])
        combined = tf.concat([duration_features, inputs['genre_Action'], inputs['genre_Adventure'], 
                              inputs['genre_Animation'], inputs['budget'], inputs['revenue']], axis=1)
                              
        return self.rating_prediction(combined)

# Experiment with model architectures
# For simplicity, this code does not explore multiple architectures as requested. You are encouraged to experiment with different layers and structures.

model = MovieRatingModel()

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=[tf.keras.metrics.RootMeanSquaredError()])

# Train the model
model.fit(train_ds, epochs=10, validation_data=test_ds)

# Evaluate the model on the test set
model.evaluate(test_ds)

# Due to the synthetic nature of this example, the actual performance may vary. 
# It's recommended to experiment with different model configurations, feature selections, 
# and preprocessing steps to improve the model's accuracy.

# Potential areas for improvement include: 
# - Including more features or embedding categorical features for better representation.
# - Hyperparameter tuning to find the optimal model configuration. 
# - Utilizing more complex models or ensemble methods to capture non-linear relationships.
# - Adding regularization to prevent overfitting.

# Remember to adjust paths, feature selections, and model configurations as per your specific dataset and problem statement.
"""

In [71]:
# using prompt_error to define a prompt to resolve the error encountered
prompt2 = prompt_error(prev_response, err)
msg_2 = get_resp_oai(prompt2,"gpt-4-0125-preview")
display(Code(msg_2, language='python'))

---

It looks like you've encountered two issues: a `SettingWithCopyWarning` warning from Pandas and a `NotImplementedError` from TensorFlow Recommenders due to the missing implementation of the `compute_loss` method in the `MovieRatingModel`. Here's how you can address both issues:

For the Pandas warning, it's best to perform operations on a DataFrame copy to avoid unintentional edits to your original DataFrame or to use the `.loc` method to ensure changes are made explicitly. For the TensorFlow error, you'll need to implement the `compute_loss` method within your model class. However, since we are directly using a Sequential model inside a class that inherits from `tfrs.models.Model`, you don't actually need a custom `compute_loss` method; instead, you can rely on built-in methods. Here's how you can adjust your code:

```python
import pandas as pd
import tensorflow as tf
import tensorflow_recommenders as tfrs

# Load the dataset
df = pd.read_csv('final_merged_df.csv')  # Update to your dataset's correct path

# Basic preprocessing
df['duration'].fillna(df['duration'].mean(), inplace=True)

features = ['duration', 'Genre_Action', 'Genre_Adventure', 'Genre_Animation', 'budget', 'revenue', 'imdb_score']
df_model = df[features].copy()  # This is where the copy is made to avoid SettingWithCopyWarning

# Normalize continuous features
for feature in ['duration', 'budget', 'revenue']:
    df_model[feature] = (df_model[feature] - df_model[feature].mean()) / df_model[feature].std()

# Split the dataset
train = df_model.sample(frac=0.8, random_state=42)
test = df_model.drop(train.index)

# Convert to TensorFlow datasets
train_ds = tf.data.Dataset.from_tensor_slices((dict(train.drop('imdb_score', axis=1)), train['imdb_score']))
test_ds = tf.data.Dataset.from_tensor_slices((dict(test.drop('imdb_score', axis=1)), test['imdb_score']))

train_ds = train_ds.batch(32).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.batch(32).prefetch(tf.data.AUTOTUNE)

# Model definition
class MovieRatingModel(tfrs.Model):  # Directly inherit from tfrs.Model

    def __init__(self):
        super().__init__()
        self.movie_model = tf.keras.Sequential([
          tf.keras.layers.Dense(64, activation='relu'),
          tf.keras.layers.Dense(32, activation='relu')
        ])
        self.rating_model = tf.keras.Sequential([
          tf.keras.layers.Dense(64, activation='relu'),
          tf.keras.layers.Dense(1)
        ])
        
        # Rating task
        self.task = tfrs.tasks.Ranking(
            loss=tf.keras.losses.MeanSquaredError(),
            metrics=[tf.keras.metrics.RootMeanSquaredError()]
        )

    def call(self, features):
        # We choose to ignore the user features in this model for simplicity's sake.
        movie_embeddings = self.movie_model(features)
        return self.rating_model(movie_embeddings)

    def compute_loss(self, features, training=False):
        labels = features.pop('imdb_score')
        rating_predictions = self(features)

        # The task computes the loss and the metrics.
        return self.task(labels=labels, predictions=rating_predictions)

# Compile and fit the model
model = MovieRatingModel()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))

model.fit(train_ds, epochs=10, validation_data=test_ds)

# Evaluate
model.evaluate(test_ds)
```

This code corrects the original implementation issues:

1. Resolves the `SettingWithCopyWarning` by creating a copy of the DataFrame slice with `.copy()`.
2. Uses directly inherited `tfrs.Model` and implements a `compute_loss` method that fits in with the TensorFlow Recommenders Systems (TFRS) framework logic, incorporating the ranking task as part of the model. This approach removes the `NotImplementedError` caused by the missing `compute_loss` implementation.

---

In [72]:
# using helper_extraction_prompt to transform the response string into an executable format
prompt3 = helper_extraction_prompt(msg_2)
msg_3 = get_resp_oai(prompt3,"gpt-4-0125-preview")
display(Code(msg_3, language='python'))

---

```python
# It looks like you've encountered two issues: a `SettingWithCopyWarning` warning from Pandas and a `NotImplementedError` from TensorFlow Recommenders 
# due to the missing implementation of the `compute_loss` method in the `MovieRatingModel`. Here's how you can address both issues:
# 
# For the Pandas warning, it's best to perform operations on a DataFrame copy to avoid unintentional edits to your original DataFrame or 
# to use the `.loc` method to ensure changes are made explicitly. For the TensorFlow error, you'll need to implement the `compute_loss` method 
# within your model class. However, since we are directly using a Sequential model inside a class that inherits from `tfrs.models.Model`, 
# you don't actually need a custom `compute_loss` method; instead, you can rely on built-in methods. Here's how you can adjust your code:

import pandas as pd
import tensorflow as tf
import tensorflow_recommenders as tfrs

# Load the dataset
df = pd.read_csv('final_merged_df.csv')  # Update to your dataset's correct path

# Basic preprocessing
df['duration'].fillna(df['duration'].mean(), inplace=True)

features = ['duration', 'Genre_Action', 'Genre_Adventure', 'Genre_Animation', 'budget', 'revenue', 'imdb_score']
df_model = df[features].copy()  # This is where the copy is made to avoid SettingWithCopyWarning

# Normalize continuous features
for feature in ['duration', 'budget', 'revenue']:
    df_model[feature] = (df_model[feature] - df_model[feature].mean()) / df_model[feature].std()

# Split the dataset
train = df_model.sample(frac=0.8, random_state=42)
test = df_model.drop(train.index)

# Convert to TensorFlow datasets
train_ds = tf.data.Dataset.from_tensor_slices((dict(train.drop('imdb_score', axis=1)), train['imdb_score']))
test_ds = tf.data.Dataset.from_tensor_slices((dict(test.drop('imdb_score', axis=1)), test['imdb_score']))

train_ds = train_ds.batch(32).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.batch(32).prefetch(tf.data.AUTOTUNE)

# Model definition
class MovieRatingModel(tfrs.Model):  # Directly inherit from tfrs.Model

    def __init__(self):
        super().__init__()
        self.movie_model = tf.keras.Sequential([
          tf.keras.layers.Dense(64, activation='relu'),
          tf.keras.layers.Dense(32, activation='relu')
        ])
        self.rating_model = tf.keras.Sequential([
          tf.keras.layers.Dense(64, activation='relu'),
          tf.keras.layers.Dense(1)
        ])
        
        # Rating task
        self.task = tfrs.tasks.Ranking(
            loss=tf.keras.losses.MeanSquaredError(),
            metrics=[tf.keras.metrics.RootMeanSquaredError()]
        )

    def call(self, features):
        # We choose to ignore the user features in this model for simplicity's sake.
        movie_embeddings = self.movie_model(features)
        return self.rating_model(movie_embeddings)

    def compute_loss(self, features, training=False):
        labels = features.pop('imdb_score')
        rating_predictions = self(features)

        # The task computes the loss and the metrics.
        return self.task(labels=labels, predictions=rating_predictions)

# Compile and fit the model
model = MovieRatingModel()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))

model.fit(train_ds, epochs=10, validation_data=test_ds)

# Evaluate
model.evaluate(test_ds)

# This code corrects the original implementation issues:
# 1. Resolves the `SettingWithCopyWarning` by creating a copy of the DataFrame slice with `.copy()`.
# 2. Uses directly inherited `tfrs.Model` and implements a `compute_loss` method that fits in with the TensorFlow Recommenders Systems (TFRS) framework logic, 
# incorporating the ranking task as part of the model. This approach removes the `NotImplementedError` caused by the missing `compute_loss` implementation.
```

---

In [75]:
exec(msg_3[len("---\n\n```python\n"):len(msg_3)-len("\n```\n\n---")])

Epoch 1/10


AttributeError: in user code:

    File "/usr/local/lib/python3.11/site-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.11/site-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.11/site-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.11/site-packages/tensorflow_recommenders/models/base.py", line 68, in train_step
        loss = self.compute_loss(inputs, training=True)
    File "<string>", line 63, in compute_loss
        

    AttributeError: 'tuple' object has no attribute 'pop'


In [79]:
err = """ 
---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
Cell In[75], line 1
----> 1 exec(msg_3[len("---\n\n```python\n"):len(msg_3)-len("\n```\n\n---")])

File <string>:73

File /usr/local/lib/python3.11/site-packages/keras/src/utils/traceback_utils.py:70, in filter_traceback.<locals>.error_handler(*args, **kwargs)
     67     filtered_tb = _process_traceback_frames(e.__traceback__)
     68     # To get the full stack trace, call:
     69     # `tf.debugging.disable_traceback_filtering()`
---> 70     raise e.with_traceback(filtered_tb) from None
     71 finally:
     72     del filtered_tb

File /tmp/__autograph_generated_filejo7rd7xr.py:15, in outer_factory.<locals>.inner_factory.<locals>.tf__train_function(iterator)
     13 try:
     14     do_return = True
---> 15     retval_ = ag__.converted_call(ag__.ld(step_function), (ag__.ld(self), ag__.ld(iterator)), None, fscope)
     16 except:
     17     do_return = False

File /usr/local/lib/python3.11/site-packages/tensorflow_recommenders/models/base.py:68, in Model.train_step(self, inputs)
     65 Custom train step using the `compute_loss` method.
     67 with tf.GradientTape() as tape:
---> 68   loss = self.compute_loss(inputs, training=True)
     70   # Handle regularization losses as well.
     71   regularization_loss = sum(self.losses)

File <string>:63, in compute_loss(self, features, training)

AttributeError: in user code:

    File "/usr/local/lib/python3.11/site-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.11/site-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.11/site-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.11/site-packages/tensorflow_recommenders/models/base.py", line 68, in train_step
        loss = self.compute_loss(inputs, training=True)
    File "<string>", line 63, in compute_loss
        

    AttributeError: 'tuple' object has no attribute 'pop' 
"""
prev_response = """
# It looks like you've encountered two issues: a `SettingWithCopyWarning` warning from Pandas and a `NotImplementedError` from TensorFlow Recommenders 
# due to the missing implementation of the `compute_loss` method in the `MovieRatingModel`. Here's how you can address both issues:
# 
# For the Pandas warning, it's best to perform operations on a DataFrame copy to avoid unintentional edits to your original DataFrame or 
# to use the `.loc` method to ensure changes are made explicitly. For the TensorFlow error, you'll need to implement the `compute_loss` method 
# within your model class. However, since we are directly using a Sequential model inside a class that inherits from `tfrs.models.Model`, 
# you don't actually need a custom `compute_loss` method; instead, you can rely on built-in methods. Here's how you can adjust your code:

import pandas as pd
import tensorflow as tf
import tensorflow_recommenders as tfrs

# Load the dataset
df = pd.read_csv('final_merged_df.csv')  # Update to your dataset's correct path

# Basic preprocessing
df['duration'].fillna(df['duration'].mean(), inplace=True)

features = ['duration', 'Genre_Action', 'Genre_Adventure', 'Genre_Animation', 'budget', 'revenue', 'imdb_score']
df_model = df[features].copy()  # This is where the copy is made to avoid SettingWithCopyWarning

# Normalize continuous features
for feature in ['duration', 'budget', 'revenue']:
    df_model[feature] = (df_model[feature] - df_model[feature].mean()) / df_model[feature].std()

# Split the dataset
train = df_model.sample(frac=0.8, random_state=42)
test = df_model.drop(train.index)

# Convert to TensorFlow datasets
train_ds = tf.data.Dataset.from_tensor_slices((dict(train.drop('imdb_score', axis=1)), train['imdb_score']))
test_ds = tf.data.Dataset.from_tensor_slices((dict(test.drop('imdb_score', axis=1)), test['imdb_score']))

train_ds = train_ds.batch(32).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.batch(32).prefetch(tf.data.AUTOTUNE)

# Model definition
class MovieRatingModel(tfrs.Model):  # Directly inherit from tfrs.Model

    def __init__(self):
        super().__init__()
        self.movie_model = tf.keras.Sequential([
          tf.keras.layers.Dense(64, activation='relu'),
          tf.keras.layers.Dense(32, activation='relu')
        ])
        self.rating_model = tf.keras.Sequential([
          tf.keras.layers.Dense(64, activation='relu'),
          tf.keras.layers.Dense(1)
        ])
        
        # Rating task
        self.task = tfrs.tasks.Ranking(
            loss=tf.keras.losses.MeanSquaredError(),
            metrics=[tf.keras.metrics.RootMeanSquaredError()]
        )

    def call(self, features):
        # We choose to ignore the user features in this model for simplicity's sake.
        movie_embeddings = self.movie_model(features)
        return self.rating_model(movie_embeddings)

    def compute_loss(self, features, training=False):
        labels = features.pop('imdb_score')
        rating_predictions = self(features)

        # The task computes the loss and the metrics.
        return self.task(labels=labels, predictions=rating_predictions)

# Compile and fit the model
model = MovieRatingModel()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))

model.fit(train_ds, epochs=10, validation_data=test_ds)

# Evaluate
model.evaluate(test_ds)

# This code corrects the original implementation issues:
# 1. Resolves the `SettingWithCopyWarning` by creating a copy of the DataFrame slice with `.copy()`.
# 2. Uses directly inherited `tfrs.Model` and implements a `compute_loss` method that fits in with the TensorFlow Recommenders Systems (TFRS) framework logic, 
# incorporating the ranking task as part of the model. This approach removes the `NotImplementedError` caused by the missing `compute_loss` implementation.
"""

In [80]:
# using prompt_error to define a prompt to resolve the error encountered
prompt4 = prompt_error(prev_response, err)
msg_4 = get_resp_oai(prompt4,"gpt-4-0125-preview")
display(Code(msg_4, language='python'))

---

It seems the issue arises from how the data is passed to the `compute_loss` method. The error message you shared suggests that the `features` variable, expected to be a dictionary or a similar object from which you can `pop` the 'imdb_score' key, is actually being received as a tuple. This mismatch typically happens when the dataset provided to the TensorFlow model does not match the expected shape or structure.

Let's adjust the code to ensure the data is prepared correctly for the model. We'll update the way the dataset is created to explicitly define a structure that separates the features from the labels right before they are batched. This structure will prevent the tuple issue when accessing the data in the `compute_loss` method:

```python
import pandas as pd
import tensorflow as tf
import tensorflow_recommenders as tfrs

# Load the dataset
df = pd.read_csv('final_merged_df.csv')  # Update to your dataset's correct path

# Basic preprocessing
df['duration'].fillna(df['duration'].mean(), inplace=True)

features = ['duration', 'Genre_Action', 'Genre_Adventure', 'Genre_Animation', 'budget', 'revenue', 'imdb_score']
df_model = df[features].copy()  # This is where the copy is made to avoid SettingWithCopyWarning

# Normalize continuous features
for feature in ['duration', 'budget', 'revenue']:
    df_model[feature] = (df_model[feature] - df_model[feature].mean()) / df_model[feature].std()

# Split the dataset
train = df_model.sample(frac=0.8, random_state=42)
test = df_model.drop(train.index)

# Correctly structure the dataset for TensorFlow
def map_func(features, label):
    return features, label

train_features = {name: np.array(value) for name, value in train.drop('imdb_score', axis=1).items()}
train_labels = np.array(train['imdb_score'])
test_features = {name: np.array(value) for name, value in test.drop('imdb_score', axis=1).items()}
test_labels = np.array(test['imdb_score'])

train_ds = tf.data.Dataset.from_tensor_slices((train_features, train_labels)).map(map_func)
test_ds = tf.data.Dataset.from_tensor_slices((test_features, test_labels)).map(map_func)

train_ds = train_ds.batch(32).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.batch(32).prefetch(tf.data.AUTOTUNE)

# Model definition remains the same
class MovieRatingModel(tfrs.Model):

    def __init__(self):
        super().__init__()
        self.movie_model = tf.keras.Sequential([
          tf.keras.layers.Dense(64, activation='relu'),
          tf.keras.layers.Dense(32, activation='relu')
        ])
        self.rating_model = tf.keras.Sequential([
          tf.keras.layers.Dense(64, activation='relu'),
          tf.keras.layers.Dense(1)
        ])
        self.task = tfrs.tasks.Ranking(
            loss=tf.keras.losses.MeanSquaredError(),
            metrics=[tf.keras.metrics.RootMeanSquaredError()]
        )

    def call(self, features):
        movie_embeddings = self.movie_model(features)
        return self.rating_model(movie_embeddings)

    def compute_loss(self, features, training=False):
        # Adjusted to expect a tuple as input
        features, labels = features  # Unpack the tuple
        rating_predictions = self.call(features)

        return self.task(labels=labels, predictions=rating_predictions)

# Compile and fit the model
model = MovieRatingModel()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))

model.fit(train_ds, epochs=10, validation_data=test_ds)

# Evaluate
model.evaluate(test_ds)
```

This modification ensures the `train_ds` and `test_ds` datasets are structured correctly, addressing the root cause of the AttributeError you encountered. By mapping the dataset into the correct format `(features, labels)` before batching, we ensure that the model receives the input in the expected structure, allowing the `compute_loss` method to work as intended.

---

In [81]:
# using helper_extraction_prompt to transform the response string into an executable format
prompt5 = helper_extraction_prompt(msg_4)
msg_5 = get_resp_oai(prompt5,"gpt-4-0125-preview")
display(Code(msg_5, language='python'))

---

```python
# It seems the issue arises from how the data is passed to the `compute_loss` method. The error message you shared suggests that the `features` variable, expected to be a dictionary or a similar object from which you can `pop` the 'imdb_score' key, is actually being received as a tuple. This mismatch typically happens when the dataset provided to the TensorFlow model does not match the expected shape or structure.
# 
# Let's adjust the code to ensure the data is prepared correctly for the model. We'll update the way the dataset is created to explicitly define a structure that separates the features from the labels right before they are batched. This structure will prevent the tuple issue when accessing the data in the `compute_loss` method:

import pandas as pd
import tensorflow as tf
import tensorflow_recommenders as tfrs
import numpy as np

# Load the dataset
df = pd.read_csv('final_merged_df.csv')  # Update to your dataset's correct path

# Basic preprocessing
df['duration'].fillna(df['duration'].mean(), inplace=True)

features = ['duration', 'Genre_Action', 'Genre_Adventure', 'Genre_Animation', 'budget', 'revenue', 'imdb_score']
df_model = df[features].copy()  # This is where the copy is made to avoid SettingWithCopyWarning

# Normalize continuous features
for feature in ['duration', 'budget', 'revenue']:
    df_model[feature] = (df_model[feature] - df_model[feature].mean()) / df_model[feature].std()

# Split the dataset
train = df_model.sample(frac=0.8, random_state=42)
test = df_model.drop(train.index)

# Correctly structure the dataset for TensorFlow
def map_func(features, label):
    return features, label

train_features = {name: np.array(value) for name, value in train.drop('imdb_score', axis=1).items()}
train_labels = np.array(train['imdb_score'])
test_features = {name: np.array(value) for name, value in test.drop('imdb_score', axis=1).items()}
test_labels = np.array(test['imdb_score'])

train_ds = tf.data.Dataset.from_tensor_slices((train_features, train_labels)).map(map_func)
test_ds = tf.data.Dataset.from_tensor_slices((test_features, test_labels)).map(map_func)

train_ds = train_ds.batch(32).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.batch(32).prefetch(tf.data.AUTOTUNE)

# Model definition remains the same
class MovieRatingModel(tfrs.Model):

    def __init__(self):
        super().__init__()
        self.movie_model = tf.keras.Sequential([
          tf.keras.layers.Dense(64, activation='relu'),
          tf.keras.layers.Dense(32, activation='relu')
        ])
        self.rating_model = tf.keras.Sequential([
          tf.keras.layers.Dense(64, activation='relu'),
          tf.keras.layers.Dense(1)
        ])
        self.task = tfrs.tasks.Ranking(
            loss=tf.keras.losses.MeanSquaredError(),
            metrics=[tf.keras.metrics.RootMeanSquaredError()]
        )

    def call(self, features):
        movie_embeddings = self.movie_model(features)
        return self.rating_model(movie_embeddings)

    def compute_loss(self, features, training=False):
        # Adjusted to expect a tuple as input
        features, labels = features  # Unpack the tuple
        rating_predictions = self.call(features)

        return self.task(labels=labels, predictions=rating_predictions)

# Compile and fit the model
model = MovieRatingModel()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))

model.fit(train_ds, epochs=10, validation_data=test_ds)

# Evaluate
model.evaluate(test_ds)

# This modification ensures the `train_ds` and `test_ds` datasets are structured correctly, addressing the root cause of the AttributeError you encountered. By mapping the dataset into the correct format `(features, labels)` before batching, we ensure that the model receives the input in the expected structure, allowing the `compute_loss` method to work as intended.
```

---

In [83]:
exec(msg_5[len("---\n\n```python\n"):len(msg_5)-len("\n```\n\n---")])

Epoch 1/10


ValueError: in user code:

    File "/usr/local/lib/python3.11/site-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.11/site-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.11/site-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.11/site-packages/tensorflow_recommenders/models/base.py", line 68, in train_step
        loss = self.compute_loss(inputs, training=True)
    File "<string>", line 67, in compute_loss
        
    File "<string>", line 61, in call
        
    File "/usr/local/lib/python3.11/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/usr/local/lib/python3.11/site-p

#### Human intervention required to resolve all errors

In [93]:
import pandas as pd
import tensorflow as tf
import tensorflow_recommenders as tfrs
import numpy as np

# Load the dataset
df = pd.read_csv('final_merged_df.csv')  

# Basic preprocessing
df['duration'].fillna(df['duration'].mean(), inplace=True)

features = ['duration', 'Genre_Action', 'Genre_Adventure', 'Genre_Animation', 'budget', 'revenue', 'imdb_score']
df_model = df[features].copy()  # This is where the copy is made to avoid SettingWithCopyWarning

# Normalize continuous features
for feature in ['duration', 'budget', 'revenue']:
    df_model[feature] = (df_model[feature] - df_model[feature].mean()) / df_model[feature].std()

# Split the dataset
train = df_model.sample(frac=0.8, random_state=42)
test = df_model.drop(train.index)

# Correctly structure the dataset for TensorFlow
def map_func(features, label):
    return features, label

train_features = {name: np.array(value) for name, value in train.drop('imdb_score', axis=1).items()}
train_labels = np.array(train['imdb_score'])
test_features = {name: np.array(value) for name, value in test.drop('imdb_score', axis=1).items()}
test_labels = np.array(test['imdb_score'])

train_ds = tf.data.Dataset.from_tensor_slices((train_features, train_labels)).map(map_func)
test_ds = tf.data.Dataset.from_tensor_slices((test_features, test_labels)).map(map_func)

train_ds = train_ds.batch(32).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.batch(32).prefetch(tf.data.AUTOTUNE)

class MovieRatingModel(tfrs.Model):

    def __init__(self):
        super().__init__()
        self.movie_model = tf.keras.Sequential([
            tf.keras.layers.Dense(64, activation='relu'),  # Adjusted input shape
            tf.keras.layers.Dense(32, activation='relu')
        ])
        self.rating_model = tf.keras.Sequential([
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(1)
        ])
        self.task = tfrs.tasks.Ranking(
            loss=tf.keras.losses.MeanSquaredError(),
            metrics=[tf.keras.metrics.RootMeanSquaredError()]
        )

    def call(self, features):
        
        # [ modified bit of code ] #
        duration = tf.expand_dims(features['duration'], axis=-1)
        genre_action = tf.expand_dims(tf.cast(features['Genre_Action'], tf.float32), axis=-1)
        genre_adventure = tf.expand_dims(tf.cast(features['Genre_Adventure'], tf.float32), axis=-1)
        genre_animation = tf.expand_dims(tf.cast(features['Genre_Animation'], tf.float32), axis=-1)
        budget = tf.expand_dims(features['budget'], axis=-1)
        revenue = tf.expand_dims(features['revenue'], axis=-1)
        
        concatenated_features = tf.concat([duration, genre_action, genre_adventure, genre_animation, budget, revenue], axis=1)
        # [ modified bit of code ] #
        
        movie_embeddings = self.movie_model(concatenated_features)
        
        return self.rating_model(movie_embeddings)

    def compute_loss(self, features, training=False):
        features, labels = features
        rating_predictions = self(features)

        return self.task(labels=labels, predictions=rating_predictions)

# Compile and fit the model
model = MovieRatingModel()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))

model.fit(train_ds, epochs=10, validation_data=test_ds)

# Evaluate
model.evaluate(test_ds)

Epoch 1/10
104/104 [==============================] - 2s 6ms/step - root_mean_squared_error: 3.9364 - loss: 15.3092 - regularization_loss: 0.0000e+00 - total_loss: 15.3092 - val_root_mean_squared_error: 1.7594 - val_loss: 2.5595 - val_regularization_loss: 0.0000e+00 - val_total_loss: 2.5595
Epoch 2/10
104/104 [==============================] - 0s 3ms/step - root_mean_squared_error: 1.4347 - loss: 2.0362 - regularization_loss: 0.0000e+00 - total_loss: 2.0362 - val_root_mean_squared_error: 1.0751 - val_loss: 1.0504 - val_regularization_loss: 0.0000e+00 - val_total_loss: 1.0504
Epoch 3/10
104/104 [==============================] - 0s 3ms/step - root_mean_squared_error: 1.0130 - loss: 1.0184 - regularization_loss: 0.0000e+00 - total_loss: 1.0184 - val_root_mean_squared_error: 0.9839 - val_loss: 1.1619 - val_regularization_loss: 0.0000e+00 - val_total_loss: 1.1619
Epoch 4/10
104/104 [==============================] - 0s 3ms/step - root_mean_squared_error: 0.9528 - loss: 0.9020 - regularizat

[0.9660033583641052, 1.2177084684371948, 0, 1.2177084684371948]

In [95]:
df['imdb_score'].unique()

array([7.9, 7.1, 6.8, 8.5, 6.6, 6.2, 7.8, 7.5, 6.9, 6.1, 6.7, 7.3, 6.5,
       7.2, 8.1, 7. , 7.7, 8.2, 5.9, 6. , 5.7, 6.3, 5.6, 8.3, 8. , 8.4,
       5.8, 5.4, 9. , 4.8, 5.2, 7.6, 4.5, 6.4, 5.5, 8.6, 8.8, 5.1, 7.4,
       4.2, 5. , 4.9, 3.7, 5.3, 4.3, 3.8, 4.4, 3.3, 2.2, 8.9, 8.7, 4.6,
       2.4, 3.4, 4.1, 4.7, 3. , 3.6, 1.7, 4. , 2.7, 2. , 3.5, 9.3, 2.9,
       3.9, 2.8, 2.3, 1.9, 3.1, 1.6, 2.5, 2.1, 9.2, 3.2, 9.1])

- In this case, the RMSE value of approximately 0.966 on the test dataset signifies that, on average, the model's predictions deviate from the actual IMDb scores by around 0.966 points.
- Interpreting this value in the context of IMDb scores, where ratings typically range from 1 to 10, a RMSE of 0.966 indicates that the model's predictions are reasonably accurate. It implies that the majority of the model's predictions are within approximately 1 point of the actual IMDb scores. Therefore, the lower the RMSE, the higher the accuracy of the model's predictions. In summary, a RMSE of 0.966 suggests that the model is performing well in predicting IMDb scores based on the given features.

### Conclusions
The model shows promising performance with decreasing training and validation RMSE, indicating effective learning and generalisation. Consistent reduction in loss during training suggests minimized prediction errors. Lack of regularisation loss suggests the model avoids overfitting. The test dataset's RMSE of approximately 0.966 signifies accurate predictions with a reasonable margin of error. Overall, the model demonstrates robust performance in predicting IMDb scores based on the provided features, with most predictions within 1 point of actual scores. This implies that the choosen LLM illustrates a promising potential to design effective NN arhitectures for specified tasks.

# Learning Rate

In [62]:
api_key= 'wx2BQHdz8yBJw318NlkHGG3PfvlFM0l0d315rmH2lv8'

In [63]:
def get_resp_oai(input_text, model):
    url = "https://llm.api.ai8.io/query_llm"
    data = {
        # Specify the model that you want to use
        "model": model,
        "messages": [
                    {"role": "system", "content": "You act as a highly intelligent system. Your job is to analyse the movie data and predict the rating scores of movies considering various movie features"},
                    {"role": "user", "content": input_text}
        ]
    }
    headers = {'Authorization': api_key}
    response = requests.post(url, json=data, headers=headers)
    if response.status_code == 200:
        response_data = json.loads(response.content)
        model_response = extract_message_oai(response_data)
        return model_response
    else:
        return {"statusCode": response.status_code, "body": response.content}

def extract_message_oai(response_data):
    message_content = response_data.get("choices", [])[0].get("message", {}).get("content", "")
    # format the extracted message as markdown
    markdown_content = "---\n\n" + message_content + "\n\n---"
    return markdown_content

In [64]:
df= pd.read_csv('final_merged_df.csv')

Running the model with memory

In [65]:
import pandas as pd
import tensorflow as tf
import tensorflow_recommenders as tfrs
import numpy as np
import pandas as pd
# Basic preprocessing
df['duration'].fillna(df['duration'].mean(), inplace=True)

features = ['duration', 'Genre_Action', 'Genre_Adventure', 'Genre_Animation', 'budget', 'revenue', 'imdb_score']
df_model = df[features].copy()  # This is where the copy is made to avoid SettingWithCopyWarning

# Normalize continuous features
for feature in ['duration', 'budget', 'revenue']:
    df_model[feature] = (df_model[feature] - df_model[feature].mean()) / df_model[feature].std()

# Split the dataset
train = df_model.sample(frac=0.8, random_state=42)
test = df_model.drop(train.index)

# Correctly structure the dataset for TensorFlow
def map_func(features, label):
    return features, label

train_features = {name: np.array(value) for name, value in train.drop('imdb_score', axis=1).items()}
train_labels = np.array(train['imdb_score'])
test_features = {name: np.array(value) for name, value in test.drop('imdb_score', axis=1).items()}
test_labels = np.array(test['imdb_score'])

train_ds = tf.data.Dataset.from_tensor_slices((train_features, train_labels)).map(map_func)
test_ds = tf.data.Dataset.from_tensor_slices((test_features, test_labels)).map(map_func)

train_ds = train_ds.batch(32).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.batch(32).prefetch(tf.data.AUTOTUNE)

class MovieRatingModel(tfrs.Model):

    def __init__(self):
        super().__init__()
        self.movie_model = tf.keras.Sequential([
            tf.keras.layers.Dense(64, activation='relu'),  # Adjusted input shape
            tf.keras.layers.Dense(32, activation='relu')
        ])
        self.rating_model = tf.keras.Sequential([
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(1)
        ])
        self.task = tfrs.tasks.Ranking(
            loss=tf.keras.losses.MeanSquaredError(),
            metrics=[tf.keras.metrics.RootMeanSquaredError()]
        )

    def call(self, features):
        
        # [ modified bit of code ] #
        duration = tf.expand_dims(features['duration'], axis=-1)
        genre_action = tf.expand_dims(tf.cast(features['Genre_Action'], tf.float32), axis=-1)
        genre_adventure = tf.expand_dims(tf.cast(features['Genre_Adventure'], tf.float32), axis=-1)
        genre_animation = tf.expand_dims(tf.cast(features['Genre_Animation'], tf.float32), axis=-1)
        budget = tf.expand_dims(features['budget'], axis=-1)
        revenue = tf.expand_dims(features['revenue'], axis=-1)
        
        concatenated_features = tf.concat([duration, genre_action, genre_adventure, genre_animation, budget, revenue], axis=1)
        # [ modified bit of code ] #
        
        movie_embeddings = self.movie_model(concatenated_features)
        
        return self.rating_model(movie_embeddings)

    def compute_loss(self, features, training=False):
        features, labels = features
        rating_predictions = self(features)

        return self.task(labels=labels, predictions=rating_predictions)

# Compile and fit the model
model = MovieRatingModel()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))

model.fit(train_ds, epochs=10, validation_data=test_ds)

Epoch 1/10


/usr/local/lib/python3.11/site-packages/keras/src/layers/layer.py:361: UserWarning: `build()` was called on layer 'movie_rating_model_11', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


104/104 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 13.9802 - regularization_loss: 0.0000e+00 - root_mean_squared_error: 4.9454 - total_loss: 13.9802 - val_loss: 3.0186 - val_regularization_loss: 0.0000e+00 - val_root_mean_squared_error: 1.8108 - val_total_loss: 3.0186
Epoch 2/10
104/104 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 2.5001 - regularization_loss: 0.0000e+00 - root_mean_squared_error: 1.7607 - total_loss: 2.5001 - val_loss: 1.1705 - val_regularization_loss: 0.0000e+00 - val_root_mean_squared_error: 1.2201 - val_total_loss: 1.1705
Epoch 3/10
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.1821 - regularization_loss: 0.0000e+00 - root_mean_squared_error: 1.1679 - total_loss: 1.1821 - val_loss: 1.1696 - val_regularization_loss: 0.0000e+00 - val_root_mean_squared_error: 0.9974 - val_total_loss: 1.1696
Epoch 4/10
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.9278 - regularization_loss: 0.0000e+00 - root_mean_squared_error: 0.9794 - total_loss: 0.9278 - val_loss: 1.2281 - val_

In [66]:
print(df_model.head()) 

   duration  Genre_Action  Genre_Adventure  Genre_Animation    budget  \
0  3.133589             1                1                0  4.851404   
1  2.725783             1                1                0  6.343649   
2  1.774236             1                1                0  5.040895   
3  2.499224             1                0                0  5.159328   
4  1.049248             1                1                0  5.396192   

     revenue  imdb_score  
0  15.796650         7.9  
1   5.092535         7.1  
2   4.621912         6.8  
3   5.818689         8.5  
4   1.126834         6.6  


In [92]:
def define_prompt_learning_rate():
    prompt = """
Task Description:
Provide me with the full code that modifies the existing model to adjust the learning rate in the optimizer to 0.05. This change aims to observe the impact of a higher learning rate on the model's training performance and prediction accuracy for movie ratings.

Current Setup:
- The model uses TensorFlow and TensorFlow Recommenders libraries.
- The current learning rate in the Adam optimizer is set at 0.001.

Required Modification:
- Provide the full code that increases the learning rate in the Adam optimizer from 0.001 to 0.05. Ensure that your response separates code from explanatory text.

Objectives:
- Implement the learning rate change with the rest of the code.
- Observe and document the effects on model training and accuracy.
- Keep the model architecture unchanged.
- Ensure that code is separated from text.

Expected Deliverables:
1. Adjust the following Python code to apply the new learning rate and document the changes inline. Provide the python code seperately from the explanations. Make sure that the code combines the full model.
2. Consider that the df is already loaded in the environment.


This is the already established model:
```python
# Basic preprocessing
df['duration'].fillna(df['duration'].mean(), inplace=True)

features = ['duration', 'Genre_Action', 'Genre_Adventure', 'Genre_Animation', 'budget', 'revenue', 'imdb_score']
df_model = df[features].copy()  # This is where the copy is made to avoid SettingWithCopyWarning

# Normalize continuous features
for feature in ['duration', 'budget', 'revenue']:
    df_model[feature] = (df_model[feature] - df_model[feature].mean()) / df_model[feature].std()

# Split the dataset
train = df_model.sample(frac=0.8, random_state=42)
test = df_model.drop(train.index)

# Correctly structure the dataset for TensorFlow
def map_func(features, label):
    return features, label

train_features = {name: np.array(value) for name, value in train.drop('imdb_score', axis=1).items()}
train_labels = np.array(train['imdb_score'])
test_features = {name: np.array(value) for name, value in test.drop('imdb_score', axis=1).items()}
test_labels = np.array(test['imdb_score'])

train_ds = tf.data.Dataset.from_tensor_slices((train_features, train_labels)).map(map_func)
test_ds = tf.data.Dataset.from_tensor_slices((test_features, test_labels)).map(map_func)

train_ds = train_ds.batch(32).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.batch(32).prefetch(tf.data.AUTOTUNE)

class MovieRatingModel(tfrs.Model):

    def __init__(self):
        super().__init__()
        self.movie_model = tf.keras.Sequential([
            tf.keras.layers.Dense(64, activation='relu'),  # Adjusted input shape
            tf.keras.layers.Dense(32, activation='relu')
        ])
        self.rating_model = tf.keras.Sequential([
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(1)
        ])
        self.task = tfrs.tasks.Ranking(
            loss=tf.keras.losses.MeanSquaredError(),
            metrics=[tf.keras.metrics.RootMeanSquaredError()]
        )

    def call(self, features):
        
        # [ modified bit of code ] #
        duration = tf.expand_dims(features['duration'], axis=-1)
        genre_action = tf.expand_dims(tf.cast(features['Genre_Action'], tf.float32), axis=-1)
        genre_adventure = tf.expand_dims(tf.cast(features['Genre_Adventure'], tf.float32), axis=-1)
        genre_animation = tf.expand_dims(tf.cast(features['Genre_Animation'], tf.float32), axis=-1)
        budget = tf.expand_dims(features['budget'], axis=-1)
        revenue = tf.expand_dims(features['revenue'], axis=-1)
        
        concatenated_features = tf.concat([duration, genre_action, genre_adventure, genre_animation, budget, revenue], axis=1)
        # [ modified bit of code ] #
        
        movie_embeddings = self.movie_model(concatenated_features)
        
        return self.rating_model(movie_embeddings)

    def compute_loss(self, features, training=False):
        features, labels = features
        rating_predictions = self(features)

        return self.task(labels=labels, predictions=rating_predictions)

# Compile and fit the model
model = MovieRatingModel()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))

model.fit(train_ds, epochs=10, validation_data=test_ds)

# Evaluate
model.evaluate(test_ds)

- Documenting observations:
 The increased learning rate is intended to test faster convergence capabilities
 Careful monitoring of model performance metrics is crucial to assess the benefits or drawbacks of this adjustment.
"""
    return prompt

In [93]:
# function to convert response string into an executable format: prompt 3, 5
def helper_extraction_prompt(resp_str):
    return f"Can you transform this string: {resp_str} into an executable string by extracting the python code and keeping the rest of the information as comments?Skip introductory text. Please place comment symbols where nessesary.Keep in mind that the given response string will be executed in a python code chunk using the function :'exec(given_response_string)."

In [94]:
prompt_lr_1 = define_prompt_learning_rate()
msg_lr_1 = get_resp_oai(prompt_lr_1,"gpt-4-0125-preview")
display(Markdown("<div style='color: #34568B;'>\n\n" + msg_lr_1))

<div style='color: #34568B;'>

---

To modify the given model to use a higher learning rate of 0.05, you need to adjust the `optimizer` parameter in the `model.compile()` step. Here's how the updated section of the code looks:

```python
# Compile and fit the model with an adjusted learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.05))

model.fit(train_ds, epochs=10, validation_data=test_ds)

# Evaluate
model.evaluate(test_ds)
```

### Full Modified Code:

```python
import tensorflow as tf
import tensorflow_recommenders as tfrs
import numpy as np
import pandas as pd

# Assume df is already loaded in your environment
# Basic preprocessing
df['duration'].fillna(df['duration'].mean(), inplace=True)

features = ['duration', 'Genre_Action', 'Genre_Adventure', 'Genre_Animation', 'budget', 'revenue', 'imdb_score']
df_model = df[features].copy()  # This is where the copy is made to avoid SettingWithCopyWarning

# Normalize continuous features
for feature in ['duration', 'budget', 'revenue']:
    df_model[feature] = (df_model[feature] - df_model[feature].mean()) / df_model[feature].std()

# Split the dataset
train = df_model.sample(frac=0.8, random_state=42)
test = df_model.drop(train.index)

def map_func(features, label):
    return features, label

train_features = {name: np.array(value) for name, value in train.drop('imdb_score', axis=1).items()}
train_labels = np.array(train['imdb_score'])
test_features = {name: np.array(value) for name, value in test.drop('imdb_score', axis=1).items()}
test_labels = np.array(test['imdb_score'])

train_ds = tf.data.Dataset.from_tensor_slices((train_features, train_labels)).map(map_func)
test_ds = tf.data.Dataset.from_tensor_slices((test_features, test_labels)).map(map_func)

train_ds = train_ds.batch(32).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.batch(32).prefetch(tf.data.AUTOTUNE)

class MovieRatingModel(tfrs.Model):

    def __init__(self):
        super().__init__()
        self.movie_model = tf.keras.Sequential([
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(32, activation='relu')
        ])
        self.rating_model = tf.keras.Sequential([
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(1)
        ])
        self.task = tfrs.tasks.Ranking(
            loss=tf.keras.losses.MeanSquaredError(),
            metrics=[tf.keras.metrics.RootMeanSquaredError()]
        )

    def call(self, features):
        duration = tf.expand_dims(features['duration'], axis=-1)
        genre_action = tf.expand_dims(tf.cast(features['Genre_Action'], tf.float32), axis=-1)
        genre_adventure = tf.expand_dims(tf.cast(features['Genre_Adventure'], tf.float32), axis=-1)
        genre_animation = tf.expand_dims(tf.cast(features['Genre_Animation'], tf.float32), axis=-1)
        budget = tf.expand_dims(features['budget'], axis=-1)
        revenue = tf.expand_dims(features['revenue'], axis=-1)
        
        concatenated_features = tf.concat([duration, genre_action, genre_adventure, genre_animation, budget, revenue], axis=1)
        
        movie_embeddings = self.movie_model(concatenated_features)
        
        return self.rating_model(movie_embeddings)

    def compute_loss(self, features, training=False):
        features, labels = features
        rating_predictions = self(features)

        return self.task(labels=labels, predictions=rating_predictions)

# Compile and fit the model with an adjusted learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.05))

model.fit(train_ds, epochs=10, validation_data=test_ds)

# Evaluate
model.evaluate(test_ds)
```

### Observing Changes:

When you modify the learning rate to 0.05, it's essential to monitor:

- **Training Loss and Validation Loss**: A higher learning rate can lead to faster convergence, but it might also overshoot the minimum, leading to potentially higher loss after initial improvement.
- **Accuracy Metrics**: In this case, RMSE (root mean squared error) would be a critical metric. Observe whether it decreases more quickly, stagnates, or even increases due to the higher learning rate.
- **Overfitting**: Ensure that the model isn't just memorizing the training data. A significant gap between training and validation metrics might indicate overfitting.

If you encounter negative impacts, consider using techniques such as learning rate schedules or decay to adjust the learning rate dynamically during training.

---

In [95]:
prompt_lr_1= helper_extraction_prompt(msg_lr_1)
msg_lr_1 = get_resp_oai(prompt_lr_1,"gpt-4-0125-preview")
display(Code(msg_lr_1, language='python'))

---

```python
# Compile and fit the model with an adjusted learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.05))

model.fit(train_ds, epochs=10, validation_data=test_ds)

# Evaluate
model.evaluate(test_ds)

"""
### Full Modified Code:
"""
import tensorflow as tf
import tensorflow_recommenders as tfrs
import numpy as np
import pandas as pd

# Assume df is already loaded in your environment
# Basic preprocessing
df['duration'].fillna(df['duration'].mean(), inplace=True)

features = ['duration', 'Genre_Action', 'Genre_Adventure', 'Genre_Animation', 'budget', 'revenue', 'imdb_score']
df_model = df[features].copy()  # This is where the copy is made to avoid SettingWithCopyWarning

# Normalize continuous features
for feature in ['duration', 'budget', 'revenue']:
    df_model[feature] = (df_model[feature] - df_model[feature].mean()) / df_model[feature].std()

# Split the dataset
train = df_model.sample(frac=0.8, random_state=42)
test = df_model.drop(train.index)

def map_func(features, label):
    return features, label

train_features = {name: np.array(value) for name, value in train.drop('imdb_score', axis=1).items()}
train_labels = np.array(train['imdb_score'])
test_features = {name: np.array(value) for name, value in test.drop('imdb_score', axis=1).items()}
test_labels = np.array(test['imdb_score'])

train_ds = tf.data.Dataset.from_tensor_slices((train_features, train_labels)).map(map_func)
test_ds = tf.data.Dataset.from_tensor_slices((test_features, test_labels)).map(map_func)

train_ds = train_ds.batch(32).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.batch(32).prefetch(tf.data.AUTOTUNE)

class MovieRatingModel(tfrs.Model):

    def __init__(self):
        super().__init__()
        self.movie_model = tf.keras.Sequential([
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(32, activation='relu')
        ])
        self.rating_model = tf.keras.Sequential([
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(1)
        ])
        self.task = tfrs.tasks.Ranking(
            loss=tf.keras.losses.MeanSquaredError(),
            metrics=[tf.keras.metrics.RootMeanSquaredError()]
        )

    def call(self, features):
        duration = tf.expand_dims(features['duration'], axis=-1)
        genre_action = tf.expand_dims(tf.cast(features['Genre_Action'], tf.float32), axis=-1)
        genre_adventure = tf.expand_dims(tf.cast(features['Genre_Adventure'], tf.float32), axis=-1)
        genre_animation = tf.expand_dims(tf.cast(features['Genre_Animation'], tf.float32), axis=-1)
        budget = tf.expand_dims(features['budget'], axis=-1)
        revenue = tf.expand_dims(features['revenue'], axis=-1)
        
        concatenated_features = tf.concat([duration, genre_action, genre_adventure, genre_animation, budget, revenue], axis=1)
        
        movie_embeddings = self.movie_model(concatenated_features)
        
        return self.rating_model(movie_embeddings)

    def compute_loss(self, features, training=False):
        features, labels = features
        rating_predictions = self(features)

        return self.task(labels=labels, predictions=rating_predictions)

model = MovieRatingModel()

# Compile and fit the model with an adjusted learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.05))

model.fit(train_ds, epochs=10, validation_data=test_ds)

# Evaluate
model.evaluate(test_ds)

"""
### Observing Changes:

When you modify the learning rate to 0.05, it's essential to monitor:

- **Training Loss and Validation Loss**: A higher learning rate can lead to faster convergence, but it might also overshoot the minimum, leading to potentially higher loss after initial improvement.
- **Accuracy Metrics**: In this case, RMSE (root mean squared error) would be a critical metric. Observe whether it decreases more quickly, stagnates, or even increases due to the higher learning rate.
- **Overfitting**: Ensure

In [96]:
exec(msg_lr_1[len("---\n\n```python\n"):len(msg_lr_1)-len("\n```\n\n---")])

Epoch 1/10


/usr/local/lib/python3.11/site-packages/keras/src/layers/layer.py:361: UserWarning: `build()` was called on layer 'movie_rating_model_13', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


104/104 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 1.9596 - regularization_loss: 0.0000e+00 - root_mean_squared_error: 1.9747 - total_loss: 1.9596 - val_loss: 1.2938 - val_regularization_loss: 0.0000e+00 - val_root_mean_squared_error: 1.0181 - val_total_loss: 1.2938
Epoch 2/10
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.0257 - regularization_loss: 0.0000e+00 - root_mean_squared_error: 1.0363 - total_loss: 1.0257 - val_loss: 1.3845 - val_regularization_loss: 0.0000e+00 - val_root_mean_squared_error: 1.0161 - val_total_loss: 1.3845
Epoch 3/10
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.9991 - regularization_loss: 0.0000e+00 - root_mean_squared_error: 1.0058 - total_loss: 0.9991 - val_loss: 1.2558 - val_regularization_loss: 0.0000e+00 - val_root_mean_squared_error: 1.0092 - val_total_loss: 1.2558
Epoch 4/10
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.9777 - regularization_loss: 0.0000e+00 - root_mean_squared_error: 0.9908 - total_loss: 0.9777 - val_loss: 1.2619 - val_re

/usr/local/lib/python3.11/site-packages/keras/src/layers/layer.py:361: UserWarning: `build()` was called on layer 'movie_rating_model_14', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


104/104 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 3.4858 - regularization_loss: 0.0000e+00 - root_mean_squared_error: 2.8410 - total_loss: 3.4858 - val_loss: 1.1050 - val_regularization_loss: 0.0000e+00 - val_root_mean_squared_error: 1.0091 - val_total_loss: 1.1050
Epoch 2/10
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.9839 - regularization_loss: 0.0000e+00 - root_mean_squared_error: 0.9966 - total_loss: 0.9839 - val_loss: 1.1209 - val_regularization_loss: 0.0000e+00 - val_root_mean_squared_error: 1.0077 - val_total_loss: 1.1209
Epoch 3/10
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.9658 - regularization_loss: 0.0000e+00 - root_mean_squared_error: 0.9866 - total_loss: 0.9658 - val_loss: 1.1267 - val_regularization_loss: 0.0000e+00 - val_root_mean_squared_error: 1.0401 - val_total_loss: 1.1267
Epoch 4/10
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.9250 - regularization_loss: 0.0000e+00 - root_mean_squared_error: 0.9716 - total_loss: 0.9250 - val_loss: 1.1386 - val_re

# Hyperparameter Tuning: Random Search

In [6]:
api_key= 'H_ClMkEoDcpQkDU4iYaTd3UNMuUUdPDbeQbDVHMnGms'

In [7]:
#Function to communicate with the LLM
def get_resp_oai(input_text, model):
    url = "https://llm.api.ai8.io/query_llm"
    data = {
        # Specify the model that you want to use
        "model": model,
        "messages": [
                    {"role": "system", "content": "You act as a highly intelligent system. Your job is to analyse the movie data and predict the rating scores of movies considering various movie features"},
                    {"role": "user", "content": input_text}
        ]
    }
    headers = {'Authorization': api_key}
    response = requests.post(url, json=data, headers=headers)
    if response.status_code == 200:
        response_data = json.loads(response.content)
        model_response = extract_message_oai(response_data)
        return model_response
    else:
        return {"statusCode": response.status_code, "body": response.content}

def extract_message_oai(response_data):
    message_content = response_data.get("choices", [])[0].get("message", {}).get("content", "")
    # format the extracted message as markdown
    markdown_content = "---\n\n" + message_content + "\n\n---"
    return markdown_content

In [8]:
df= pd.read_csv('final_merged_df.csv')

In [9]:
#Load and preprocess the dataframe
import pandas as pd
import tensorflow as tf
import tensorflow_recommenders as tfrs
import numpy as np
import pandas as pd
# Basic preprocessing
df['duration'].fillna(df['duration'].mean(), inplace=True)

features = ['duration', 'Genre_Action', 'Genre_Adventure', 'Genre_Animation', 'budget', 'revenue', 'imdb_score']
df_model = df[features].copy()  # This is where the copy is made to avoid SettingWithCopyWarning

print(df.columns)

# Normalize continuous features
for feature in ['duration', 'budget', 'revenue']:
    df_model[feature] = (df_model[feature] - df_model[feature].mean()) / df_model[feature].std()

# Split the dataset
train = df_model.sample(frac=0.8, random_state=42)
test = df_model.drop(train.index)

# Correctly structure the dataset for TensorFlow
def map_func(features, label):
    return features, label

train_features = {name: np.array(value) for name, value in train.drop('imdb_score', axis=1).items()}
train_labels = np.array(train['imdb_score'])
test_features = {name: np.array(value) for name, value in test.drop('imdb_score', axis=1).items()}
test_labels = np.array(test['imdb_score'])

train_ds = tf.data.Dataset.from_tensor_slices((train_features, train_labels)).map(map_func)
test_ds = tf.data.Dataset.from_tensor_slices((test_features, test_labels)).map(map_func)

train_ds = train_ds.batch(32).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.batch(32).prefetch(tf.data.AUTOTUNE)

class MovieRatingModel(tfrs.Model):

    def __init__(self):
        super().__init__()
        self.movie_model = tf.keras.Sequential([
            tf.keras.layers.Dense(64, activation='relu'),  # Adjusted input shape
            tf.keras.layers.Dense(32, activation='relu')
        ])
        self.rating_model = tf.keras.Sequential([
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(1)
        ])
        self.task = tfrs.tasks.Ranking(
            loss=tf.keras.losses.MeanSquaredError(),
            metrics=[tf.keras.metrics.RootMeanSquaredError()]
        )

    def call(self, features):
        
        # [ modified bit of code ] #
        duration = tf.expand_dims(features['duration'], axis=-1)
        genre_action = tf.expand_dims(tf.cast(features['Genre_Action'], tf.float32), axis=-1)
        genre_adventure = tf.expand_dims(tf.cast(features['Genre_Adventure'], tf.float32), axis=-1)
        genre_animation = tf.expand_dims(tf.cast(features['Genre_Animation'], tf.float32), axis=-1)
        budget = tf.expand_dims(features['budget'], axis=-1)
        revenue = tf.expand_dims(features['revenue'], axis=-1)

        concatenated_features = tf.concat([duration, genre_action, genre_adventure, genre_animation, budget, revenue], axis=1)
        # [ modified bit of code ] #
        
        movie_embeddings = self.movie_model(concatenated_features)
        
        return self.rating_model(movie_embeddings)

    def compute_loss(self, features, training=False):
        features, labels = features
        rating_predictions = self(features)

        return self.task(labels=labels, predictions=rating_predictions)

# Compile and fit the model
model = MovieRatingModel()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))

model.fit(train_ds, epochs=10, validation_data=test_ds)


Index(['Unnamed: 0', 'director_name', 'duration', 'actor_2_name', 'genres_x',
       'actor_1_name', 'movie_title', 'actor_3_name', 'movie_imdb_link',
       'language', 'country', 'title_year', 'imdb_score', 'budget',
       'original_title', 'revenue', 'tagline', 'Genre_Action',
       'Genre_Adventure', 'Genre_Animation', 'Genre_Biography', 'Genre_Comedy',
       'Genre_Crime', 'Genre_Documentary', 'Genre_Drama', 'Genre_Family',
       'Genre_Fantasy', 'Genre_Film-Noir', 'Genre_History', 'Genre_Horror',
       'Genre_Music', 'Genre_Musical', 'Genre_Mystery', 'Genre_News',
       'Genre_Romance', 'Genre_Sci-Fi', 'Genre_Sport', 'Genre_Thriller',
       'Genre_War', 'Genre_Western', 'duration_category', 'rating_category',
       'budget_category', 'revenue_category', 'budget_revenue_ratio',
       'log_budget', 'log_revenue', 'log_title_year', 'sqr_root_duration'],
      dtype='object')
Epoch 1/10
104/104 [==============================] - 1s 5ms/step - root_mean_squared_error: 3.9034 

In [10]:
print(df_model.head()) 

   duration  Genre_Action  Genre_Adventure  Genre_Animation    budget  \
0  3.133589             1                1                0  4.851404   
1  2.725783             1                1                0  6.343649   
2  1.774236             1                1                0  5.040895   
3  2.499224             1                0                0  5.159328   
4  1.049248             1                1                0  5.396192   

     revenue  imdb_score  
0  15.796650         7.9  
1   5.092535         7.1  
2   4.621912         6.8  
3   5.818689         8.5  
4   1.126834         6.6  


In [11]:
#Explain the Random Search task and desired outcomes to the LLM
def define_prompt_random_search():
    prompt = """
Task Description:

Provide me with the full code that enhances the existing neural network model with a Random Search. This change aims to determine the optimal number of neurons in the first dense layer and optimal learning rate for predicting the IMDB_Score movie ratings.

Current Setup:
- The model uses TensorFlow and TensorFlow Recommenders libraries.
- The model should import keras_tuner using !pip
- The model preprocesses the features by including one hot encoded genres and normalized numeric features to input into the neural network.
- There are two parts to the neural network: one concatenates movie features and the other predicts movie ratings.

Required Modification:
- Provide the full code that performs hyperparameter tuning using keras_tuner's Random Search. The output should include the optimal number of neurons in the first dense layer and learning rate for the optimizer. Ensure that your response separates code from explanatory text.

Objectives:
- Perform hyperparameter tuning to explore different configurations of the number of neurons in the first dense layer and the learning rate.
- The hyperparameters identified should optimize the accuracy of the model. 
- Keep the model architecture unchanged.
- Ensure that code is separated from text.
- Any text output should be commented out using #

Expected Deliverables:
1. Adjust the following Python code to perform Random Search hyperparameter tuning and document the changes inline. Provide the python code seperately from the explanations. Make sure that the code combines the full model.
2. Consider that the df is already loaded in the environment.


This is the already established model:
```python
# Basic preprocessing
df['duration'].fillna(df['duration'].mean(), inplace=True)

features = ['duration', 'Genre_Action', 'Genre_Adventure', 'Genre_Animation', 'budget', 'revenue', 'imdb_score']
df_model = df[features].copy()  # This is where the copy is made to avoid SettingWithCopyWarning

# Normalize continuous features
for feature in ['duration', 'budget', 'revenue']:
    df_model[feature] = (df_model[feature] - df_model[feature].mean()) / df_model[feature].std()

# Split the dataset
train = df_model.sample(frac=0.8, random_state=42)
test = df_model.drop(train.index)

# Correctly structure the dataset for TensorFlow
def map_func(features, label):
    return features, label

train_features = {name: np.array(value) for name, value in train.drop('imdb_score', axis=1).items()}
train_labels = np.array(train['imdb_score'])
test_features = {name: np.array(value) for name, value in test.drop('imdb_score', axis=1).items()}
test_labels = np.array(test['imdb_score'])

train_ds = tf.data.Dataset.from_tensor_slices((train_features, train_labels)).map(map_func)
test_ds = tf.data.Dataset.from_tensor_slices((test_features, test_labels)).map(map_func)

train_ds = train_ds.batch(32).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.batch(32).prefetch(tf.data.AUTOTUNE)

class MovieRatingModel(tfrs.Model):

    def __init__(self):
        super().__init__()
        self.movie_model = tf.keras.Sequential([
            tf.keras.layers.Dense(64, activation='relu'),  # Adjusted input shape
            tf.keras.layers.Dense(32, activation='relu')
        ])
        self.rating_model = tf.keras.Sequential([
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(1)
        ])
        self.task = tfrs.tasks.Ranking(
            loss=tf.keras.losses.MeanSquaredError(),
            metrics=[tf.keras.metrics.RootMeanSquaredError()]
        )

    def call(self, features):
        
        # [ modified bit of code ] #
        duration = tf.expand_dims(features['duration'], axis=-1)
        genre_action = tf.expand_dims(tf.cast(features['Genre_Action'], tf.float32), axis=-1)
        genre_adventure = tf.expand_dims(tf.cast(features['Genre_Adventure'], tf.float32), axis=-1)
        genre_animation = tf.expand_dims(tf.cast(features['Genre_Animation'], tf.float32), axis=-1)
        budget = tf.expand_dims(features['budget'], axis=-1)
        revenue = tf.expand_dims(features['revenue'], axis=-1)
        
        concatenated_features = tf.concat([duration, genre_action, genre_adventure, genre_animation, budget, revenue], axis=1)
        # [ modified bit of code ] #
        
        movie_embeddings = self.movie_model(concatenated_features)
        
        return self.rating_model(movie_embeddings)

    def compute_loss(self, features, training=False):
        features, labels = features
        rating_predictions = self(features)

        return self.task(labels=labels, predictions=rating_predictions)

# Compile and fit the model
model = MovieRatingModel()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))

model.fit(train_ds, epochs=10, validation_data=test_ds)

# Evaluate
model.evaluate(test_ds)

- Documenting observations:
 The hyperparameter tuning is intended to optimize the model's performance.
 Careful monitoring of model performance metrics is crucial to assess the benefits or drawbacks of this adjustment.
"""
    return prompt
    

In [12]:
# Function to convert LLM response string into an executable format
def helper_extraction_prompt(resp_str):
    return f"Transform this string: {resp_str} to extract python code. Do not provide any string text or explanations or else it will be an inconvenience and will need to be removed manually. If string text is ouputted, please use # to comment it out. The output will be used directly as Python code :'exec(given_response_string)."

In [13]:
# Function to prompt the model for error resolution 
def prompt_error(prev_response, err):
    return f"Given that the code you provided: {prev_response} gives this error: {err}, help me resolve it by changing the code you provided where nessesary, your reposne should be in an executable code format"

In [14]:
prompt_rand_search_1 = define_prompt_random_search()
msg_rand_search_1 = get_resp_oai(prompt_rand_search_1,"gpt-4-0125-preview")
display(Markdown("<div style='color: #34568B;'>\n\n" + msg_rand_search_1))

KeyboardInterrupt: 

In [ ]:
#Execute the code
exec(msg_rand_search_1[len("---\n\n```python\n"):len(msg_rand_search_1)-len("\n```\n\n---")])

In [ ]:
#Explain the error received to the LLM and the adjustements required
err = """ 

Traceback (most recent call last):

  File /usr/local/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3550 in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)

  Cell In[51], line 1
    exec(msg_rand_search_1[len("---\n\n```python\n"):len(msg_rand_search_1)-len("\n```\n\n---")])

  File <string>:1
    hyperparameter tuning using keras_tuner's Random Search for optimizing the number of neurons in the first dense layer and the learning rate for the optimizer, you need to make some modifications and additions to your existing code. Here's how you can incorporate keras_tuner into your workflow:
    ^
IndentationError: unexpected indent
"""
 
prev_response = """
# It's recommended to perform hyperparameter tuning using Random Search and to test different dense layer neuron counts and learning rates to make optimal predictions on IMDB_score. 
"""

     

In [ ]:
# Use prompt_error to define a prompt to resolve the error encountered
prompt_rand_search_2 = prompt_error(prev_response, err)
msg_rand_search_2 = get_resp_oai(prompt_rand_search_2,"gpt-4-0125-preview")
display(Code(msg_rand_search_2, language='python'))

In [ ]:
# Use helper_extraction_prompt to transform the response string into an executable format
prompt_rand_search_3 = helper_extraction_prompt(msg_rand_search_2)
msg_rand_search_3 = get_resp_oai(prompt_rand_search_3,"gpt-4-0125-preview")
display(Code(msg_rand_search_3, language='python'))

In [ ]:
exec(msg_rand_search_3[len("---\n\n```python\n"):len(msg_rand_search_3)-len("\n```\n\n---")])

In [ ]:
#Explain error received to the LLM
err = """ 
ModuleNotFoundError                       Traceback (most recent call last)
Cell In[57], line 1
----> 1 exec(msg_rand_search_3[len("---\n\n```python\n"):len(msg_rand_search_3)-len("\n```\n\n---")])

File <string>:3

ModuleNotFoundError: No module named 'kerastuner'
"""
 
prev_response = """
# It's recommended to perform hyperparameter tuning using Random Search and to test different dense layer neuron counts and learning rates to make optimal predictions on IMDB_score. 
"""


In [ ]:
# Use prompt_error to define a prompt to resolve the error encountered
prompt_rand_search_4 = prompt_error(prev_response, err)
msg_rand_search_4 = get_resp_oai(prompt_rand_search_4,"gpt-4-0125-preview")
display(Code(msg_rand_search_4, language='python'))

In [ ]:
# Use helper_extraction_prompt to transform the response string into an executable format
prompt_rand_search_5 = helper_extraction_prompt(msg_rand_search_4)
msg_rand_search_5 = get_resp_oai(prompt_rand_search_5,"gpt-4-0125-preview")
display(Code(msg_rand_search_5, language='python'))

In [ ]:
#Execute the code
exec(msg_rand_search_5[len("---\n\n```python\n"):len(msg_rand_search_5)-len("\n```\n\n---")])

In [ ]:
#Explain the error received
err = """ 
#You are experiencing an error because the 'kerastuner' has been installed incorrectly. It should be installed by using !pip install keras-tuner.

Traceback (most recent call last):

  File /usr/local/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3550 in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)

  Cell In[51], line 1
    exec(msg_rand_search_1[len("---\n\n```python\n"):len(msg_rand_search_1)-len("\n```\n\n---")])

  File <string>:1
    hyperparameter tuning using keras_tuner's Random Search for optimizing the number of neurons in the first dense layer and the learning rate for the optimizer, you need to make some modifications and additions to your existing code. Here's how you can incorporate keras_tuner into your workflow:
    ^
IndentationError: unexpected indent
"""
 
prev_response = """
# With the exception of incorrectly importing kerastuner, the first part of the code was correct until #tuner.search. 
#The code following tuner_search should be included as a code and not a string.
#The code should also apply the tuner search to the validation data.
#After performing the hyperparameter search, the best perfoming dense layer neuron count and learning rate should be applied to perform predictions.
#After deriving the predictions, the best hyperparamater results should be printed.
#The accuracy and the classification report should print the results from the prediction model that incorporated the optimal dense layer neuron count and learning rate. 
#This was the last code used that needs to be improved using the above criteria: 

```python
import tensorflow as tf
from tensorflow import keras
from kerastuner.tuners import RandomSearch

input_shape = (10,)

def build_model(hp):
    model = keras.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    model.add(keras.layers.Dense(units=hp_units, activation='relu'))
    model.add(keras.layers.Dense(1, activation='linear'))
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss='mean_squared_error',
                  metrics=['mean_absolute_error'])

    return model

tuner = RandomSearch(
    build_model,
    objective='val_mean_absolute_error',
    max_trials=5,
    executions_per_trial=3,
    directory='my_dir',
    project_name='imdb_score_tuning')

# tuner.search_space_summary()

# X_train, y_train = ...
# X_val, y_val = ...

# tuner.search(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

# models = tuner.get_best_models(num_models=2)

# best_hyperparameters = tuner.get_best_hyperparameters()[0]
```

---

"""

In [ ]:
#Reprompt the LLM
prompt_rand_search_6 = define_prompt_random_search()
msg_rand_search_6 = get_resp_oai(prompt_rand_search_6,"gpt-4-0125-preview")
display(Markdown("<div style='color: #34568B;'>\n\n" + msg_rand_search_6))

In [ ]:
#Use helper_extraction_prompt to transform the response string into an executable format
prompt_rand_search_7 = helper_extraction_prompt(msg_rand_search_6)
msg_rand_search_7 = get_resp_oai(prompt_rand_search_5,"gpt-4-0125-preview")
display(Code(msg_rand_search_7, language='python'))

In [ ]:
#Execute the code
exec(msg_rand_search_7[len("---\n\n```python\n"):len(msg_rand_search_7)-len("\n```\n\n---")])

In [ ]:
#Define the error received 
err = """ 
ModuleNotFoundError                       Traceback (most recent call last)
Cell In[65], line 1
----> 1 exec(msg_rand_search_7[len("---\n\n```python\n"):len(msg_rand_search_7)-len("\n```\n\n---")])

File <string>:3

ModuleNotFoundError: No module named 'kerastuner'
"""
 
prev_response = """
# You need to install keras tuner using !'pip install keras-tuner' 
"""

In [ ]:
#Reprompt the LLM
prompt_rand_search_8 = define_prompt_random_search()
msg_rand_search_8 = get_resp_oai(prompt_rand_search_8,"gpt-4-0125-preview")
display(Markdown("<div style='color: #34568B;'>\n\n" + msg_rand_search_8))

In [ ]:
#Use the helper_extraction_prompt to transform the response string into an executable format
prompt_rand_search_9 = helper_extraction_prompt(msg_rand_search_8)
msg_rand_search_9 = get_resp_oai(prompt_rand_search_9,"gpt-4-0125-preview")
display(Code(msg_rand_search_9, language='python'))

In [ ]:
#Execute the message
exec(msg_rand_search_9[len("---\n\n```python\n"):len(msg_rand_search_9)-len("\n```\n\n---")])

In [ ]:
#Define the error received
err = """ 
ModuleNotFoundError                       Traceback (most recent call last)
Cell In[69], line 1
----> 1 exec(msg_rand_search_9[len("---\n\n```python\n"):len(msg_rand_search_9)-len("\n```\n\n---")])

File <string>:5

ModuleNotFoundError: No module named 'keras_tuner'

"""
 
prev_response = """
# Install !pip install keras-tuner in the first line of code.
#Do not hash or comment out the code beginning at tuner.search
#Make predictions on the test set after determining the optimal hyperparameters from Random Search
#Print the classification report and accuracy of the predictions that use the best hyperparameters found in Random Search
"""

In [ ]:
#Reprompt the LLM
prompt_rand_search_9 = define_prompt_random_search()
msg_rand_search_9 = get_resp_oai(prompt_rand_search_9,"gpt-4-0125-preview")
display(Markdown("<div style='color: #34568B;'>\n\n" + msg_rand_search_9))

In [ ]:
#Use helper_extraction_prompt to transform the response string into an executable format
prompt_rand_search_10 = helper_extraction_prompt(msg_rand_search_9)
msg_rand_search_10 = get_resp_oai(prompt_rand_search_10,"gpt-4-0125-preview")
display(Code(msg_rand_search_10, language='python'))

In [ ]:
#Execute the code
exec(msg_rand_search_10[len("---\n\n```python\n"):len(msg_rand_search_10)-len("\n```\n\n---")])

In [ ]:
err = """ 

ModuleNotFoundError                       Traceback (most recent call last)
Cell In[74], line 1
----> 1 exec(msg_rand_search_10[len("---\n\n```python\n"):len(msg_rand_search_10)-len("\n```\n\n---")])

File <string>:5

ModuleNotFoundError: No module named 'keras_tuner'
"""
 
prev_response = """
# The code will not run because not all of the correct modules and packages have been imported. It is very important to load the keras-tuner module using !pip.
#The "import keras_tuner as kt" does not suffice. In addition to importing this package, the module needs to be imported.
"""

In [ ]:
#Define using prompting error
prompt_rand_search_11 = define_prompt_random_search()
msg_rand_search_11 = get_resp_oai(prompt_rand_search_11,"gpt-4-0125-preview")
display(Markdown("<div style='color: #34568B;'>\n\n" + msg_rand_search_11))

In [ ]:
#Use helper_extraction_prompt to transform the response string into an executable format
prompt_rand_search_12 = helper_extraction_prompt(msg_rand_search_11)
msg_rand_search_12 = get_resp_oai(prompt_rand_search_12,"gpt-4-0125-preview")
display(Code(msg_rand_search_12, language='python'))

In [ ]:
#Execute the code
exec(msg_rand_search_12[len("---\n\n```python\n"):len(msg_rand_search_12)-len("\n```\n\n---")])

In [ ]:
#Define the error
err = """ 
Traceback (most recent call last):

  File /usr/local/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3550 in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)

  Cell In[80], line 1
    exec(msg_rand_search_12[len("---\n\n```python\n"):len(msg_rand_search_12)-len("\n```\n\n---")])

  File <string>:1
    !pip install keras-tuner
    ^
SyntaxError: invalid syntax
"""
 
prev_response = """
# Remove the !pip install keras-tuner 
#Do not hash or comment out the code beginning at tuner.search
#Make predictions on the test set after determining the optimal hyperparameters from Random Search
#Print the classification report and accuracy of the predictions that use the best hyperparameters found in Random Search
"""

In [ ]:
#Define using prompting error
prompt_rand_search_13 = define_prompt_random_search()
msg_rand_search_13 = get_resp_oai(prompt_rand_search_13,"gpt-4-0125-preview")
display(Markdown("<div style='color: #34568B;'>\n\n" + msg_rand_search_13))

In [ ]:
#Use helper_extraction_prompt to transform the response string into an executable format
prompt_rand_search_14 = helper_extraction_prompt(msg_rand_search_13)
msg_rand_search_14 = get_resp_oai(prompt_rand_search_14,"gpt-4-0125-preview")
display(Code(msg_rand_search_14, language='python'))

In [ ]:
#Execute the code
exec(msg_rand_search_14[len("---\n\n```python\n"):len(msg_rand_search_14)-len("\n```\n\n---")])

In [ ]:
#Define the error received 
err = """ 

Traceback (most recent call last):

  File /usr/local/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3550 in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)

  Cell In[87], line 1
    exec(msg_rand_search_14[len("---\n\n```python\n"):len(msg_rand_search_14)-len("\n```\n\n---")])

  File <string>:1
    !pip install keras-tuner
    ^
SyntaxError: invalid syntax
"""
 
prev_response = """
# Do not install the keras-tuner module, this means remove the !pip install keras-tuner code.
# Add prediction results for the model
# Add the accuracy and classification report for the model
"""

In [ ]:
#Define using prompting error
prompt_rand_search_15 = define_prompt_random_search()
msg_rand_search_15 = get_resp_oai(prompt_rand_search_15,"gpt-4-0125-preview")
display(Markdown("<div style='color: #34568B;'>\n\n" + msg_rand_search_15))

In [ ]:
#Use helper_extraction_prompt to transform the response string into an executable format
prompt_rand_search_16 = helper_extraction_prompt(msg_rand_search_15)
msg_rand_search_16 = get_resp_oai(prompt_rand_search_16,"gpt-4-0125-preview")
display(Code(msg_rand_search_16, language='python'))

In [ ]:
#Execute the code
exec(msg_rand_search_16[len("---\n\n```python\n"):len(msg_rand_search_16)-len("\n```\n\n---")])

In [ ]:
#Define the error received 
err = """ 
ValueError                                Traceback (most recent call last)
Cell In[92], line 1
----> 1 exec(msg_rand_search_16[len("---\n\n```python\n"):len(msg_rand_search_16)-len("\n```\n\n---")])

File <string>:57

File /usr/local/lib/python3.11/site-packages/keras_tuner/src/tuners/randomsearch.py:164, in RandomSearch.__init__(self, hypermodel, objective, max_trials, seed, hyperparameters, tune_new_entries, allow_new_entries, max_retries_per_trial, max_consecutive_failed_trials, **kwargs)
    150 def __init__(
    151     self,
    152     hypermodel=None,
   (...)
    161     **kwargs
    162 ):
    163     self.seed = seed
--> 164     oracle = RandomSearchOracle(
    165         objective=objective,
    166         max_trials=max_trials,
    167         seed=seed,
    168         hyperparameters=hyperparameters,
    169         tune_new_entries=tune_new_entries,
    170         allow_new_entries=allow_new_entries,
    171         max_retries_per_trial=max_retries_per_trial,
    172         max_consecutive_failed_trials=max_consecutive_failed_trials,
    173     )
    174     super().__init__(oracle, hypermodel, **kwargs)

File /usr/local/lib/python3.11/site-packages/keras_tuner/src/tuners/randomsearch.py:73, in RandomSearchOracle.__init__(self, objective, max_trials, seed, hyperparameters, allow_new_entries, tune_new_entries, max_retries_per_trial, max_consecutive_failed_trials)
     62 def __init__(
     63     self,
     64     objective=None,
   (...)
     71     max_consecutive_failed_trials=3,
     72 ):
---> 73     super().__init__(
     74         objective=objective,
     75         max_trials=max_trials,
     76         hyperparameters=hyperparameters,
     77         tune_new_entries=tune_new_entries,
     78         allow_new_entries=allow_new_entries,
     79         seed=seed,
     80         max_retries_per_trial=max_retries_per_trial,
     81         max_consecutive_failed_trials=max_consecutive_failed_trials,
     82     )

File /usr/local/lib/python3.11/site-packages/keras_tuner/src/engine/oracle.py:314, in Oracle.__init__(self, objective, max_trials, hyperparameters, allow_new_entries, tune_new_entries, seed, max_retries_per_trial, max_consecutive_failed_trials)
    303 def __init__(
    304     self,
    305     objective=None,
   (...)
    312     max_consecutive_failed_trials=3,
    313 ):
--> 314     self.objective = obj_module.create_objective(objective)
    315     self.max_trials = max_trials
    316     if not hyperparameters:

File /usr/local/lib/python3.11/site-packages/keras_tuner/src/engine/objective.py:155, in create_objective(objective)
    148     error_msg = (
    149         'Could not infer optimization direction ("min" or "max") '
    150         'for unknown metric "{obj}". Please specify the objective  as'
    151         "a `keras_tuner.Objective`, for example `keras_tuner.Objective("
    152         '"{obj}", direction="min")`.'
    153     )
    154     error_msg = error_msg.format(obj=objective)
--> 155     raise ValueError(error_msg)
    156 return Objective(name=objective, direction=direction)

ValueError: Could not infer optimization direction ("min" or "max") for unknown metric "val_root_mean_squared_error". Please specify the objective  asa `keras_tuner.Objective`, for example `keras_tuner.Objective("val_root_mean_squared_error", direction="min")`.
"""
 
prev_response = """
# The val_root_mean_squared_error in the Objective function needs to be specified with a direction. Either "val_root_mean_squared_error, direction = min" or "val_root_mean_squared_error, direction = "max"
# In tuner, you must define the hypermodel, hyperparameters, and objective

"""

In [ ]:
#Define using prompting error
prompt_rand_search_17 = define_prompt_random_search()
msg_rand_search_17 = get_resp_oai(prompt_rand_search_17,"gpt-4-0125-preview")
display(Markdown("<div style='color: #34568B;'>\n\n" + msg_rand_search_17))

In [ ]:
#Use helper_extraction_prompt to transform the response string into an executable format
prompt_rand_search_18 = helper_extraction_prompt(msg_rand_search_17)
msg_rand_search_18 = get_resp_oai(prompt_rand_search_18,"gpt-4-0125-preview")
display(Code(msg_rand_search_18, language='python'))

In [ ]:
#Execute the code
exec(msg_rand_search_18[len("---\n\n```python\n"):len(msg_rand_search_18)-len("\n```\n\n---")])

In [ ]:
#Define the error message
err = """ 
Traceback (most recent call last):

  File /usr/local/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3550 in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)

  Cell In[97], line 1
    exec(msg_rand_search_18[len("---\n\n```python\n"):len(msg_rand_search_18)-len("\n```\n\n---")])

  File <string>:1
    !pip install -q keras-tuner
    ^
SyntaxError: invalid syntax
"""
 
prev_response = """
# Do not install the keras-tuner module, meaning do not use the "!pip install keras-tuner" code
# The remaining code can be kept the same.
"""

In [ ]:
#Define using prompting error
prompt_rand_search_19 = define_prompt_random_search()
msg_rand_search_19 = get_resp_oai(prompt_rand_search_19,"gpt-4-0125-preview")
display(Markdown("<div style='color: #34568B;'>\n\n" + msg_rand_search_19))

In [ ]:
#Use helper_extraction_prompt to transform the response string into an executable format
prompt_rand_search_20 = helper_extraction_prompt(msg_rand_search_19)
msg_rand_search_20 = get_resp_oai(prompt_rand_search_20,"gpt-4-0125-preview")
display(Code(msg_rand_search_20, language='python'))

In [ ]:
#Execute the code
exec(msg_rand_search_20[len("---\n\n```python\n"):len(msg_rand_search_20)-len("\n```\n\n---")])

In [ ]:
#Define the error
err = """ 
Traceback (most recent call last):

  File /usr/local/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3550 in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)

  Cell In[102], line 1
    exec(msg_rand_search_20[len("---\n\n```python\n"):len(msg_rand_search_20)-len("\n```\n\n---")])

  File <string>:1
    !pip install keras-tuner
    ^
SyntaxError: invalid syntax
"""
 
prev_response = """
# Do not install the keras-tuner module.
# Do not import the keras-tuner module.
# The keras-tuner code should not be included. 
"""

In [ ]:
#Define using prompting error
prompt_rand_search_21 = define_prompt_random_search()
msg_rand_search_21 = get_resp_oai(prompt_rand_search_21,"gpt-4-0125-preview")
display(Markdown("<div style='color: #34568B;'>\n\n" + msg_rand_search_21))

In [ ]:
#Use helper_extraction_prompt to transform the response string into an executable format
prompt_rand_search_22 = helper_extraction_prompt(msg_rand_search_21)
msg_rand_search_22 = get_resp_oai(prompt_rand_search_22,"gpt-4-0125-preview")
display(Code(msg_rand_search_22, language='python'))

In [ ]:
#Execute the code 
exec(msg_rand_search_22[len("---\n\n```python\n"):len(msg_rand_search_22)-len("\n```\n\n---")])

In [13]:
#Manually updated Random Search using the same structure developed by the LLM

#Load the appropriate packages
import numpy as np
import tensorflow as tf
import tensorflow_recommenders as tfrs
from tensorflow import keras
from keras_tuner import RandomSearch, Objective
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error

# Preprocessing
df['duration'].fillna(df['duration'].mean(), inplace=True)
features = ['duration', 'Genre_Action', 'Genre_Adventure', 'Genre_Animation', 'budget', 'revenue', 'imdb_score']
df_model = df[features].copy()

# Normalize numeric variables
for feature in ['duration', 'budget', 'revenue']:
    df_model[feature] = (df_model[feature] - df_model[feature].mean()) / df_model[feature].std()

# Split the dataset
train = df_model.sample(frac=0.8, random_state=42)
test = df_model.drop(train.index)

def map_func(features, label):
    return features, label

#Define the labels and features for training and testing
train_features = {name: np.array(value) for name, value in train.drop('imdb_score', axis=1).items()}
train_labels = np.array(train['imdb_score'])
test_features = {name: np.array(value) for name, value in test.drop('imdb_score', axis=1).items()}
test_labels = np.array(test['imdb_score'])

train_ds = tf.data.Dataset.from_tensor_slices((train_features, train_labels)).map(map_func)
test_ds = tf.data.Dataset.from_tensor_slices((test_features, test_labels)).map(map_func)

train_ds = train_ds.batch(32).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.batch(32).prefetch(tf.data.AUTOTUNE)

objective = Objective('root_mean_squared_error', direction = "min")

#Construct the neural network
def build_model(hp):
    class MovieRatingModel(tfrs.Model):
        def __init__(self, hp):
            super().__init__()
            self.movie_model = tf.keras.Sequential([
                tf.keras.layers.Dense(hp.Int('units', min_value=32, max_value=512, step=32), activation='relu'),
                tf.keras.layers.Dense(32, activation='relu')
            ])
            self.rating_model = tf.keras.Sequential([
                tf.keras.layers.Dense(64, activation='relu'),
                tf.keras.layers.Dense(1)
            ])
            self.task = tfrs.tasks.Ranking(
                loss=tf.keras.losses.MeanSquaredError(),
                metrics=[tf.keras.metrics.RootMeanSquaredError()]
            )
        
        def call(self, features):
            concatenated_features = tf.concat(
                [tf.expand_dims(tf.cast(features[name], tf.float32), -1) for name in features],
                axis=1
            )
            movie_embeddings = self.movie_model(concatenated_features)
            return self.rating_model(movie_embeddings)
            
        def compute_loss(self, features, training=False):
            features, labels = features  
            rating_predictions = self(features)  
            loss = self.task(labels=labels, predictions=rating_predictions)  
            return loss + sum(self.losses)  

    model = MovieRatingModel(hp)
    model.compile(optimizer=tf.keras.optimizers.Adam(
        learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
        run_eagerly=True)

    return model

#Construct Random Search
tuner = RandomSearch(
    build_model,
    objective= objective,
    max_trials=10,
    executions_per_trial=2,
    directory='random_search',
    project_name='movie_rating_tuning'
)

tuner.search(train_ds, epochs=10, validation_data=test_ds)

best_hps = tuner.get_best_hyperparameters(num_trials=10)[0]
best_model = tuner.hypermodel.build(best_hps)

best_model.fit(train_ds, epochs=10, validation_data=test_ds)

#Transform the test sets labels 
y_true = np.concatenate([y.numpy() if y.ndim > 0 else np.array([y.numpy()]) for x, y in test_ds.unbatch()])

#Make predictions 
y_pred = best_model.predict(test_ds)
y_pred_labels = np.argmax(y_pred, axis=1) 

#Evaluate 
results = best_model.evaluate(test_ds)

#Regression Results
print("Root Mean Squared Error:", results[1])  
print("Mean Squared Error:", results[2])      

#Make predictions using the optimal hyperparameters
y_pred = best_model.predict(test_ds)
y_pred = np.squeeze(y_pred)

#Mean Squared Error
mse = mean_squared_error(y_true, y_pred)
print("Mean Squared Error:", mse)

#Best hyperparameters
print("Dense Layer Neuron Count:", best_hps.get('units'))
print("Learning Rate:", best_hps.get('learning_rate'))


Reloading Tuner from random_search/movie_rating_tuning/tuner0.json
Epoch 1/10
104/104 [==============================] - 4s 36ms/step - root_mean_squared_error: 3.4060 - loss: 11.4626 - regularization_loss: 0.0000e+00 - total_loss: 11.4626 - val_root_mean_squared_error: 1.6758 - val_loss: 1.8385 - val_regularization_loss: 0.0000e+00 - val_total_loss: 1.8385
Epoch 2/10
104/104 [==============================] - 4s 36ms/step - root_mean_squared_error: 1.3516 - loss: 1.8072 - regularization_loss: 0.0000e+00 - total_loss: 1.8072 - val_root_mean_squared_error: 1.0356 - val_loss: 1.0559 - val_regularization_loss: 0.0000e+00 - val_total_loss: 1.0559
Epoch 3/10
104/104 [==============================] - 4s 36ms/step - root_mean_squared_error: 0.9837 - loss: 0.9606 - regularization_loss: 0.0000e+00 - total_loss: 0.9606 - val_root_mean_squared_error: 0.9741 - val_loss: 1.2340 - val_regularization_loss: 0.0000e+00 - val_total_loss: 1.2340
Epoch 4/10
104/104 [==============================] - 4s 3